# DES Y6 Deep Field Exposures:  Downloads and Munging

## 1. Setup

In [1]:
import numpy as np
import pandas as pd
from scipy import interpolate
import glob
import math
import os
import subprocess
import sys
import gc

import glob
import pickle

import easyaccess as ea

import fitsio
from astropy.io import fits
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table

import tempfile

import matplotlib.pyplot as plt
 
%matplotlib inline

In [2]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## 2. User Input

### 2.1.  General User Input  

In [89]:
verbose = 1
tag_des = 'Y6A2_FINALCUT'      # Official tag for DES Y6A2_FINALCUT
tag_eff_des = 'Y6A1_FINALCUT'  # Effective tag for DES Y6A2_FINALCUT (basically, it is Y6A1_FINALCUT)
tag_decade = 'DECADE_FINALCUT' # Tag for DECADE
output_dir = '../RawData'
for_calib_dir='../for_calib'
stilts_exe='/usrdevel/dp0/dtucker/STILTS/latest/stilts'
bandList = ['u', 'g', 'r', 'i', 'z']

### 2.2. Logical Variables to Indicate which Code Cells to Run...

In [90]:
do_save_uniqExposureList = True

do_query_des_ZPs = True
do_query_decade_ZPs = True

do_query_des_exps = True
do_query_decade_exps = True

do_query_des_filepaths = True
do_query_decade_filepaths = True

do_query_des_img_skypos = True
do_query_decade_img_skypos = True

do_query_des_exp_finalcut_qa = True
do_query_decade_exp_finalcut_qa = True

do_query_des_expimage_clinks = True
do_query_decade_expimage_clinks = True

do_query_des_exp_qa = True
do_query_decade_exp_qa = True

do_query_des_img_info = True
do_query_decade_img_info = True

do_inner_des_joins = True
do_inner_decade_joins = True

do_create_des_curl_script = True
do_create_decade_curl_script = True

do_run_des_curl_script = True
do_run_decade_curl_script = True

do_concatenate_des_se_objects = True
do_concatenate_decade_se_objects = True

do_match_des_galex = True
do_match_decade_galex = True

do_match_des_gaiaedr3 = True
do_match_decade_gaiaedr3 = True

do_match_des_fgcm_psf = True
do_match_decade_fgcm_psf = True

do_match_des_fgcm_psf = True
do_match_decade_fgcm_psf = True




do_read_deep_field_lists = False

In [4]:
# Check on TMPDIR... 
tempfile.gettempdir()

'/data/des40.a/data/dtucker/TmpDir'

## 3. Useful Modules

In [5]:
# Using suggestions from Eric Neilsen's notes page:
# http://des-ops.fnal.gov:8080/notes/neilsen/pandasExamples/pandas_examples.html#orgheadline6
def catfits_to_catdf(inputFile, expnum, ccdnum):
    npcat = fitsio.read(inputFile, columns=['NUMBER','X_IMAGE','Y_IMAGE','ALPHAWIN_J2000','DELTAWIN_J2000',
                                            'FLUX_PSF','FLUXERR_PSF','FLUX_APER','FLUXERR_APER',
                                            'CLASS_STAR','SPREAD_MODEL','SPREADERR_MODEL','FLAGS',
                                            'IMAFLAGS_ISO'], ext=2)
    npcat = npcat.byteswap().newbyteorder()

    inputFileBaseName = os.path.basename(inputFile)
    #expnum = -1  # Temporary placeholder for exposure number
    #ccdnum = -1  # Temporary placeholder for CCD number
    number = npcat['NUMBER']
    x_image = npcat['X_IMAGE']
    y_image = npcat['Y_IMAGE']
    alphawin_j2000 = npcat['ALPHAWIN_J2000']
    deltawin_j2000 = npcat['DELTAWIN_J2000']
    flux_psf = npcat['FLUX_PSF']
    fluxerr_psf = npcat['FLUXERR_PSF']
    class_star = npcat['CLASS_STAR']
    spread_model = npcat['SPREAD_MODEL']
    spreaderr_model = npcat['SPREADERR_MODEL']
    flags = npcat['FLAGS']
    imaflags_iso = npcat['IMAFLAGS_ISO']

    
    flux_aper = {}
    fluxerr_aper = {}
    for i in range(12):
        aper_n = i+1
        #print npfluxaper['FLUX_APER'][0,i]
        flux_aper[aper_n] = npcat['FLUX_APER'][:,i]
        fluxerr_aper[aper_n] = npcat['FLUXERR_APER'][:,i]

    df_cat = pd.DataFrame({    'FILENAME':inputFileBaseName, \
                               'OBJECT_NUMBER':number, \
                               'X_IMAGE':x_image, \
                               'Y_IMAGE':y_image, \
                               'RA':alphawin_j2000, \
                               'DEC':deltawin_j2000, \
                               'FLUX_PSF':flux_psf, \
                               'FLUXERR_PSF':fluxerr_psf, \
                               'FLUX_APER_01':flux_aper[1],'FLUXERR_APER_01':fluxerr_aper[1], \
                               'FLUX_APER_02':flux_aper[2],'FLUXERR_APER_02':fluxerr_aper[2], \
                               'FLUX_APER_03':flux_aper[3],'FLUXERR_APER_03':fluxerr_aper[3], \
                               'FLUX_APER_04':flux_aper[4],'FLUXERR_APER_04':fluxerr_aper[4], \
                               'FLUX_APER_05':flux_aper[5],'FLUXERR_APER_05':fluxerr_aper[5], \
                               'FLUX_APER_06':flux_aper[6],'FLUXERR_APER_06':fluxerr_aper[6], \
                               'FLUX_APER_07':flux_aper[7],'FLUXERR_APER_07':fluxerr_aper[7], \
                               'FLUX_APER_08':flux_aper[8],'FLUXERR_APER_08':fluxerr_aper[8], \
                               'FLUX_APER_09':flux_aper[9],'FLUXERR_APER_09':fluxerr_aper[9], \
                               'FLUX_APER_10':flux_aper[10],'FLUXERR_APER_10':fluxerr_aper[10], \
                               'FLUX_APER_11':flux_aper[11],'FLUXERR_APER_11':fluxerr_aper[11], \
                               'FLUX_APER_12':flux_aper[12],'FLUXERR_APER_12':fluxerr_aper[12], \
                               'CLASS_STAR':class_star, \
                               'SPREAD_MODEL':spread_model, \
                               'SPREADERR_MODEL':spreaderr_model, \
                               'FLAGS':flags, \
                               'IMAFLAGS_ISO':imaflags_iso \
                               })

    df_cat['RA_WRAP'] = df_cat['RA']
    ra_wrap_mask = (df_cat.RA_WRAP >= 180.)
    df_cat[ra_wrap_mask]['RA_WRAP'] = df_cat[ra_wrap_mask]['RA_WRAP'] - 360.
    
    df_cat['EXPNUM'] = expnum
    df_cat['CCDNUM'] = ccdnum

    cols = ['RA_WRAP','FILENAME','EXPNUM','CCDNUM', 
            'OBJECT_NUMBER','X_IMAGE','Y_IMAGE','RA','DEC',
            'FLUX_PSF','FLUXERR_PSF',
            'FLUX_APER_01','FLUX_APER_02','FLUX_APER_03','FLUX_APER_04','FLUX_APER_05','FLUX_APER_06',
            'FLUX_APER_07','FLUX_APER_08','FLUX_APER_09','FLUX_APER_10','FLUX_APER_11','FLUX_APER_12',
            'FLUXERR_APER_01','FLUXERR_APER_02','FLUXERR_APER_03','FLUXERR_APER_04','FLUXERR_APER_05','FLUXERR_APER_06',
            'FLUXERR_APER_07','FLUXERR_APER_08','FLUXERR_APER_09','FLUXERR_APER_10','FLUXERR_APER_11','FLUXERR_APER_12',
            'CLASS_STAR','SPREAD_MODEL','SPREADERR_MODEL','FLAGS','IMAFLAGS_ISO']
    df_cat = df_cat[cols]

    df_cat = df_cat.sort_values(by=['RA'])

    return df_cat

## 4. Read in DES/DECADE Deep Field Exposure Lists

In [6]:
# First, list exposure list files...

%ls -lh ../for_calib

total 1.3M
-rw-rw-r-- 1 dtucker des  280 May 25 10:38 exposures_ALHAMBARA2_g_COADD_TRUTH.npy
-rw-rw-r-- 1 dtucker des  280 May 25 10:38 exposures_ALHAMBARA2_g_DEEPEST.npy
-rw-rw-r-- 1 dtucker des  200 May 25 10:38 exposures_ALHAMBARA2_g_SE_TRUTH.npy
-rw-rw-r-- 1 dtucker des  280 May 25 10:38 exposures_ALHAMBARA2_i_COADD_TRUTH.npy
-rw-rw-r-- 1 dtucker des  280 May 25 10:38 exposures_ALHAMBARA2_i_DEEPEST.npy
-rw-rw-r-- 1 dtucker des  160 May 25 10:38 exposures_ALHAMBARA2_i_SE_TRUTH.npy
-rw-rw-r-- 1 dtucker des  272 May 25 10:38 exposures_ALHAMBARA2_r_COADD_TRUTH.npy
-rw-rw-r-- 1 dtucker des  272 May 25 10:38 exposures_ALHAMBARA2_r_DEEPEST.npy
-rw-rw-r-- 1 dtucker des  176 May 25 10:38 exposures_ALHAMBARA2_r_SE_TRUTH.npy
-rw-rw-r-- 1 dtucker des  144 Jun 10 06:46 exposures_ALHAMBARA2_u_COADD_TRUTH_DESonly.npy
-rw-rw-r-- 1 dtucker des  144 Jun 10 06:46 exposures_ALHAMBARA2_u_COADD_TRUTH_DESplsDECADE.npy
-rw-rw-r-- 1 dtucker des  304 May 25 10:38 exposures_ALHAMBARA2_z_COADD_TRUTH.npy
-rw-r

In [7]:
# Next, create a list of DES/DECADE exposure list files...

inputFileNameWildcard = os.path.join(for_calib_dir,'exposures_*.npy')
inputFileNameList = glob.glob(inputFileNameWildcard)
print inputFileNameList

['../for_calib/exposures_SN-C2_u_COADD_TRUTH_DESplsDECADE.npy', '../for_calib/exposures_EDFS2_u_COADD_TRUTH_DESonly.npy', '../for_calib/exposures_EDFS5_u_COADD_TRUTH_DESonly.npy', '../for_calib/exposures_ALHAMBARA2_u_COADD_TRUTH_DESonly.npy', '../for_calib/exposures_MACS0416_u_COADD_TRUTH_DESonly.npy', '../for_calib/exposures_SN-S1_u_COADD_TRUTH_DESplsDECADE.npy', '../for_calib/exposures_SN-X3_u_COADD_TRUTH_DESplsDECADE.npy', '../for_calib/exposures_SN-E1_u_COADD_TRUTH_DESonly.npy', '../for_calib/exposures_SN-X1_u_COADD_TRUTH_DESonly.npy', '../for_calib/exposures_SN-E2_u_COADD_TRUTH_DESonly.npy', '../for_calib/exposures_SN-C3_u_COADD_TRUTH_DESplsDECADE.npy', '../for_calib/exposures_SN-C1_u_COADD_TRUTH_DESplsDECADE.npy', '../for_calib/exposures_COSMOS_u_COADD_TRUTH_DESplsDECADE.npy', '../for_calib/exposures_SN-S2_u_COADD_TRUTH_DESonly.npy', '../for_calib/exposures_SN-X3_u_COADD_TRUTH_DESonly.npy', '../for_calib/exposures_EDFS2_u_COADD_TRUTH_DESplsDECADE.npy', '../for_calib/exposures_COS

In [8]:
# Now, read in each of the DES/DECADE exposure list files, extracting the individual exposure numbers.
# Once all are read in, create a list containing only unique entries.

exposureList = []
for inputFileName in inputFileNameList:
    data = np.load(inputFileName,allow_pickle=True)
    exposureList.extend(data)
exposureSet = set(exposureList)
uniqExposureList = (list(exposureSet))
uniqExposureArray = np.sort(np.array(uniqExposureList))
uniqExposureList = (list(uniqExposureArray))
print uniqExposureList
print
print(len(uniqExposureList))

[151381, 151382, 151757, 154852, 154854, 154855, 154856, 154857, 154858, 154859, 154861, 154866, 154867, 154868, 157445, 157446, 157447, 157448, 157449, 157452, 157453, 157454, 157455, 157456, 157460, 157461, 157462, 157463, 157464, 157467, 157468, 157469, 157470, 157471, 157474, 157475, 157476, 157477, 157478, 157482, 157483, 157484, 157485, 157486, 157490, 157491, 157492, 157493, 157501, 157502, 157503, 157504, 157523, 157524, 157525, 157526, 157527, 157528, 157529, 157530, 157531, 157532, 157533, 157534, 157535, 157536, 157537, 157538, 157799, 157800, 157801, 157802, 157803, 157804, 157805, 157806, 157807, 157808, 157809, 158146, 158185, 158186, 158187, 158205, 158206, 158209, 158622, 158623, 158624, 158625, 158626, 158629, 158630, 158631, 158632, 158633, 159856, 159857, 159858, 159859, 159860, 159863, 159864, 159865, 159866, 159867, 159884, 159885, 159886, 159887, 159888, 159892, 159893, 159894, 159895, 159896, 159898, 159899, 159900, 159901, 159902, 159903, 159904, 159905, 159906,

In [9]:
# Create a Pandas Dataframe containing the list of unique EXPNUMs...
df_uniqExposureList = pd.DataFrame (uniqExposureList, columns = ['EXPNUM'])
df_uniqExposureList

,EXPNUM
0,151381
1,151382
2,151757
3,154852
4,154854
5,154855
6,154856
7,154857
8,154858
9,154859


In [10]:
# Output results to CSV file...
if do_save_uniqExposureList:
    output_file = os.path.join(for_calib_dir, 'uniqExposureList.csv')
    print output_file
    df_uniqExposureList.to_csv(output_file, index=False)

../for_calib/uniqExposureList.csv


## 4. Zeropoints and EXPNUMs from DES Y6A2 and DECADE Databases

In [11]:
%%time

# Query Y6A2_ZEROPOINT from NCSA DES db...

if do_query_des_ZPs:
    
    print 'Query image filepaths from NCSA DES db...'

    # The query...
    #  See:  https://sites.google.com/view/delve-survey/data?pli=1&authuser=1
    query = \
        """
        SELECT * 
        FROM Y6A2_ZEROPOINT
        ORDER BY EXPNUM, CCDNUM, IMAGENAME
        """
    
    print query

    # Query NCSA DES db...
    connection=ea.connect('dessci')
    df_des_zp = connection.query_to_pandas(query)
    connection.close()

    # Output results to CSV file...
    output_file = os.path.join(output_dir, 'DES.Y6A2_ZEROPOINT.csv')
    print output_file
    df_des_zp.to_csv(output_file, index=False)

Query image filepaths from NCSA DES db...

        SELECT * 
        FROM Y6A2_ZEROPOINT
        ORDER BY EXPNUM, CCDNUM, IMAGENAME
        
Connecting to DB ** dessci ** ...
../RawData/DES.Y6A2_ZEROPOINT.csv
CPU times: user 2min 38s, sys: 16.5 s, total: 2min 54s
Wall time: 3min 39s


In [12]:
# Show first 10 lines of DataFrame...
if do_query_des_ZPs:
    print df_des_zp.head(10)

                                IMAGENAME SOURCE    VERSION                                CATALOGNAME  EXPNUM  CCDNUM BAND INSERT_DATE   MAG_ZERO  SIGMA_MAG_ZERO    MAG_ONE SIGMA_MAG_ONE            TAG  FLAG
0  D00226648_g_c01_r3650p04_immasked.fits   FGCM  y6a1_v2.1  D00226648_g_c01_r3650p04_red-fullcat.fits  226648       1    g        None  32.249756        0.002242  -3.081166          None  Y6A1_FINALCUT     1
1  D00226648_g_c02_r3650p04_immasked.fits   FGCM  y6a1_v2.1  D00226648_g_c02_r3650p04_red-fullcat.fits  226648       2    g        None  32.246090        0.002330   1.655038          None  Y6A1_FINALCUT     1
2  D00226648_g_c03_r3650p04_immasked.fits   FGCM  y6a1_v2.1  D00226648_g_c03_r3650p04_red-fullcat.fits  226648       3    g        None  32.247650        0.002150   6.377914          None  Y6A1_FINALCUT     1
3  D00226648_g_c04_r3650p04_immasked.fits   FGCM  y6a1_v2.1  D00226648_g_c04_r3650p04_red-fullcat.fits  226648       4    g        None  32.248074        0.002157  

In [13]:
%%time

# Query ZPs from NCSA DECADE db...

if do_query_decade_ZPs:
    
    print 'Query ZPs from NCSA DECADE db...'

    # The query...
    #  See:  https://sites.google.com/view/delve-survey/data?pli=1&authuser=1
    query = \
        """
        SELECT * 
        FROM ZEROPOINT
        ORDER BY EXPNUM, CCDNUM, IMAGENAME
        """
    
    print query

    # Query NCSA DECADE db...
    connection=ea.connect('delve')
    df_decade_zp = connection.query_to_pandas(query)
    connection.close()

    # Output results to CSV file...
    output_file = os.path.join(output_dir, 'DECADE.ZEROPOINT.csv')
    print output_file
    df_decade_zp.to_csv(output_file, index=False)

CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 16.9 µs


In [14]:
# Show first 10 lines of DataFrame...
if do_query_decade_ZPs:
    print df_decade_zp.head(10)

In [15]:
%%time

# Query all EXPOSUREs from NCSA DESOPER db...

if do_query_des_exps: 
    
    print 'Query all EXPOSUREs from NCSA DESOPER db...'

    # The query...
    query = \
        """
        SELECT * 
        FROM PROD.EXPOSURE
        ORDER BY EXPNUM
        """
    
    print query

    # Query NCSA DESOPER db...
    connection=ea.connect('desoper')
    df_des_exp = connection.query_to_pandas(query)
    connection.close()

    # Output results to CSV file...
    output_file = os.path.join(output_dir, 'DES.EXPOSURE.csv')
    print output_file
    df_des_exp.to_csv(output_file, index=False)

Query all EXPOSUREs from NCSA DESOPER db...

        SELECT * 
        FROM PROD.EXPOSURE
        ORDER BY EXPNUM
        
Connecting to DB ** desoper ** ...
../RawData/DES.EXPOSURE.csv
CPU times: user 32.8 s, sys: 3.14 s, total: 35.9 s
Wall time: 46.2 s


In [16]:
# Show first 10 lines of DataFrame...
if do_query_des_exps:
    print df_des_exp.head(10)

                   FILENAME FILETYPE  BAND      NITE  TELEQUIN    HA  ZD  AIRMASS                             TELFOCUS OBJECT       OBSERVER      PROPID DETECTOR            DETSIZE             TELESCOPE OBSERVATORY   LATITUDE  LONGITUDE ALTITUDE TIMESYS                    DATE_OBS         TIME_OBS       MJD_OBS  EXPTIME  EXPREQ         ...           PHOTFLAG NEXTEND         TELRA         TELDEC OBSTYPE  EXPNUM PROCTYPE  PRODTYPE PROPOSER INSTRUMENT PIXSCAL1  PIXSCAL2 CAMSHUT RADESYS FILTER FILTPOS  TELSTAT       RADEG     DECDEG    TDECDEG      TRADEG CAMSYM FIELD  TILING              PROGRAM
0  DECam_com5_00133757.fits      raw  None  20120911    2000.0  None NaN      NaN   0.00, 0.00, 0.00, 0.00, 0.00, 0.00   None  Soares-Santos  2012B-9999     None  [1:29400,1:29050]  CTIO 4.0-m telescope        CTIO  -30.16606  -70.81489     2215     UTC  2012-09-11T23:50:34.581689  23:50:34.581689  56181.993456      0.0     0.0         ...               None      70   18:38:8.820  -28:09:52.801   

In [17]:
%%time

# Query all EXPOSUREs from NCSA DECADE db...

if do_query_decade_exps: 

    print 'Query all EXPOSUREs from NCSA DECADE db...'

    # The query...
    query = \
        """
        SELECT * 
        FROM EXPOSURE
        ORDER BY EXPNUM
        """
    
    print query

    # Query NCSA DESSCI db...
    connection=ea.connect('delve')
    df_decade_exp = connection.query_to_pandas(query)
    connection.close()

    # Output results to CSV file...
    output_file = os.path.join(output_dir, 'DECADE.EXPOSURE.csv')
    print output_file
    df_decade_exp.to_csv(output_file, index=False)

CPU times: user 12 µs, sys: 1e+03 ns, total: 13 µs
Wall time: 19.1 µs


In [18]:
# Show first 10 lines of DataFrame...
if do_query_decade_exps:
    print df_decade_exp.head(10)

## 5. General Info on Exposures, Images, etc. from the DESSCI and DECADE Database

### 5.a.  Paths to the images and catalogs at NCSA

**We will first work with the DESSCI db, and then we will repeat for the DECADE db.**

**DESSCI db:**

The following cell takes about 7 minutes to run...

In [19]:
%%time

# Query image filepaths from NCSA DESSCI db...

if do_query_des_filepaths:

    print 'Query image filepaths from NCSA DESSCI db...'

    # The query...
    #  See:  https://sites.google.com/view/delve-survey/data?pli=1&authuser=1
    query = \
        """
        with 
        images as 
            (select i.expnum, i.ccdnum, i.band, 
                    racmin, racmax, deccmin, deccmax, 
                    'https://desar2.cosmology.illinois.edu/DESFiles/desarchive/'||fai.path||'/'||i.filename||fai.compression as ilink 
             from Y6A2_PROCTAG t, Y6A2_IMAGE i, Y6A2_FILE_ARCHIVE_INFO fai
             where t.tag='%s' and 
                   t.pfw_attempt_id=i.pfw_attempt_id and 
                   i.filetype='red_immask' and 
                   i.filename=fai.filename 
             order by expnum, ccdnum),  
        catalogs as 
            (select c.expnum, c.ccdnum, 
                    'https://desar2.cosmology.illinois.edu/DESFiles/desarchive/'||fai.path||'/'||c.filename||fai.compression as clink 
             from Y6A2_PROCTAG t, Y6A2_CATALOG c, Y6A2_FILE_ARCHIVE_INFO fai 
             where t.tag='%s' and 
                   t.pfw_attempt_id=c.pfw_attempt_id and 
                   c.filetype='cat_finalcut' and 
                   c.filename=fai.filename 
             order by expnum, ccdnum) 
        select images.expnum as expnum, images.ccdnum as ccdnum, 
               band, racmin, racmax, deccmin, deccmax, ilink, clink 
        from images, catalogs 
        where images.expnum = catalogs.expnum and 
              images.ccdnum = catalogs.ccdnum
        order by expnum, ccdnum
        """ % (tag_eff_des, tag_eff_des)
    
    print query

    # Query NCSA DESSCI db...
    connection=ea.connect('dessci')
    df0_des = connection.query_to_pandas(query)
    connection.close()

    # Output results to CSV file...
    output_file = """DES_%s.expimage_paths.csv""" % (tag_des)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df0_des.to_csv(output_file, index=False)

Query image filepaths from NCSA DESSCI db...

        with 
        images as 
            (select i.expnum, i.ccdnum, i.band, 
                    racmin, racmax, deccmin, deccmax, 
                    'https://desar2.cosmology.illinois.edu/DESFiles/desarchive/'||fai.path||'/'||i.filename||fai.compression as ilink 
             from Y6A2_PROCTAG t, Y6A2_IMAGE i, Y6A2_FILE_ARCHIVE_INFO fai
             where t.tag='Y6A1_FINALCUT' and 
                   t.pfw_attempt_id=i.pfw_attempt_id and 
                   i.filetype='red_immask' and 
                   i.filename=fai.filename 
             order by expnum, ccdnum),  
        catalogs as 
            (select c.expnum, c.ccdnum, 
                    'https://desar2.cosmology.illinois.edu/DESFiles/desarchive/'||fai.path||'/'||c.filename||fai.compression as clink 
             from Y6A2_PROCTAG t, Y6A2_CATALOG c, Y6A2_FILE_ARCHIVE_INFO fai 
             where t.tag='Y6A1_FINALCUT' and 
                   t.pfw_attempt_id=c.pfw_attempt

In [20]:
# Show first 10 lines of DataFrame...
if do_query_des_filepaths:
    print df0_des.head(10)

   EXPNUM  CCDNUM BAND      RACMIN      RACMAX   DECCMIN   DECCMAX                                                                                                                                                  ILINK                                                                                                                                                  CLINK
0  154833       1    g  359.541352  359.840757  0.791482  0.940849  https://desar2.cosmology.illinois.edu/DESFiles/desarchive/OPS/finalcut/Y5A1/r3727/20121123/D00154833/p02/red/immask/D00154833_g_c01_r3727p02_immas...  https://desar2.cosmology.illinois.edu/DESFiles/desarchive/OPS/finalcut/Y5A1/r3727/20121123/D00154833/p02/cat/D00154833_g_c01_r3727p02_red-fullcat....
1  154833       2    g  359.851496    0.151182  0.790486  0.940536  https://desar2.cosmology.illinois.edu/DESFiles/desarchive/OPS/finalcut/Y5A1/r3727/20121123/D00154833/p02/red/immask/D00154833_g_c02_r3727p02_immas...  https://desar2.cosmology.illinois.edu/DESFi

In [21]:
%%time

# Perform inner join with Deep Field list.

if do_query_des_filepaths:

    df0_des_deep = pd.merge(df0_des, df_uniqExposureList, \
                            on=['EXPNUM'], \
                            how='inner', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df0_des_deep if colname.endswith('_y')]
    df0_des_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """DES_%s.expimage_paths.y6deep.csv""" % (tag_des)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df0_des_deep.to_csv(output_file, index=False)

../RawData/DES_Y6A2_FINALCUT.expimage_paths.y6deep.csv
CPU times: user 10 s, sys: 1.76 s, total: 11.8 s
Wall time: 19.9 s


In [22]:
# Show first 10 lines of DataFrame...
if do_query_des_filepaths:
    print df0_des_deep.head(10)

   EXPNUM  CCDNUM BAND     RACMIN     RACMAX    DECCMIN    DECCMAX                                                                                                                                                  ILINK                                                                                                                                                  CLINK
0  154852       1    z  52.130154  52.467110 -27.270366 -27.120527  https://desar2.cosmology.illinois.edu/DESFiles/desarchive/OPS/finalcut/Y5A1/r3727/20121123/D00154852/p02/red/immask/D00154852_z_c01_r3727p02_immas...  https://desar2.cosmology.illinois.edu/DESFiles/desarchive/OPS/finalcut/Y5A1/r3727/20121123/D00154852/p02/cat/D00154852_z_c01_r3727p02_red-fullcat....
1  154852       2    z  52.479072  52.815881 -27.271100 -27.121378  https://desar2.cosmology.illinois.edu/DESFiles/desarchive/OPS/finalcut/Y5A1/r3727/20121123/D00154852/p02/red/immask/D00154852_z_c02_r3727p02_immas...  https://desar2.cosmology.illinois.edu/DESFi

**DECADE db:**

The following cell takes about 9 minutes to run...

In [23]:
%%time

# Query image filepaths from NCSA DECADE db...

if do_query_decade_filepaths:

    print 'Query image filepaths from NCSA DECADE db...'

    # The query...
    #  See:  https://sites.google.com/view/delve-survey/data?pli=1&authuser=1
    query = \
        """
        with 
        images as 
            (select i.expnum, i.ccdnum, i.band, 
                    racmin, racmax, deccmin, deccmax, 
                    'https://decade.ncsa.illinois.edu/deca_archive/'||fai.path||'/'||i.filename||fai.compression as ilink 
             from decade.proctag t, decade.image i, decade.file_archive_info fai 
             where t.tag='%s' and 
                   t.pfw_attempt_id=i.pfw_attempt_id and 
                   i.filetype='red_immask' and 
                   i.filename=fai.filename 
             order by expnum, ccdnum),  
        catalogs as 
            (select c.expnum, c.ccdnum, 
                    'https://decade.ncsa.illinois.edu/deca_archive/'||fai.path||'/'||c.filename||fai.compression as clink 
             from decade.proctag t, decade.catalog c, decade.file_archive_info fai 
             where t.tag='%s' and 
                   t.pfw_attempt_id=c.pfw_attempt_id and 
                   c.filetype='cat_finalcut' and 
                   c.filename=fai.filename 
             order by expnum, ccdnum) 
        select images.expnum as expnum, images.ccdnum as ccdnum, 
               band, racmin, racmax, deccmin, deccmax, ilink, clink 
        from images, catalogs 
        where images.expnum = catalogs.expnum and 
              images.ccdnum = catalogs.ccdnum
        order by expnum, ccdnum
        """ % (tag_decade, tag_decade)
    
    print query

    # Query NCSA DECADE db...
    connection=ea.connect('delve')
    df0_decade = connection.query_to_pandas(query)
    connection.close()

    # Output results to CSV file...
    output_file = """%s.expimage_paths.csv""" % (tag_decade)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df0_decade.to_csv(output_file, index=False)

Query image filepaths from NCSA DECADE db...

        with 
        images as 
            (select i.expnum, i.ccdnum, i.band, 
                    racmin, racmax, deccmin, deccmax, 
                    'https://decade.ncsa.illinois.edu/deca_archive/'||fai.path||'/'||i.filename||fai.compression as ilink 
             from decade.proctag t, decade.image i, decade.file_archive_info fai 
             where t.tag='DECADE_FINALCUT' and 
                   t.pfw_attempt_id=i.pfw_attempt_id and 
                   i.filetype='red_immask' and 
                   i.filename=fai.filename 
             order by expnum, ccdnum),  
        catalogs as 
            (select c.expnum, c.ccdnum, 
                    'https://decade.ncsa.illinois.edu/deca_archive/'||fai.path||'/'||c.filename||fai.compression as clink 
             from decade.proctag t, decade.catalog c, decade.file_archive_info fai 
             where t.tag='DECADE_FINALCUT' and 
                   t.pfw_attempt_id=c.pfw_attempt_id and

In [24]:
# Show first 10 lines of DataFrame...
if do_query_decade_filepaths:
    print df0_decade.head(10)

   EXPNUM  CCDNUM BAND      RACMIN    RACMAX    DECCMIN    DECCMAX                                                                                                                                              ILINK                                                                                                                                       CLINK
0  145425       1    g    0.112582  0.454720 -28.873491 -28.723264  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121101-r4148/D00145425/p01/red/immask/D00145425_g_c01_r4148p01_immasked.fits.fz  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121101-r4148/D00145425/p01/cat/D00145425_g_c01_r4148p01_red-fullcat.fits
1  145425       2    g    0.466757  0.808707 -28.874450 -28.724455  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121101-r4148/D00145425/p01/red/immask/D00145425_g_c02_r4148p01_immasked.fits.fz  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121101-r414

In [25]:
%%time

# Perform inner join with Deep Field exposure list on DECADE-based expimage_paths dataframe.

if do_query_decade_filepaths:

    df0_decade_deep = pd.merge(df0_decade, df_uniqExposureList, \
                            on=['EXPNUM'], \
                            how='inner', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df0_decade_deep if colname.endswith('_y')]
    df0_decade_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """%s.expimage_paths.y6deep.csv""" % (tag_decade)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df0_decade_deep.to_csv(output_file, index=False)

../RawData/DECADE_FINALCUT.expimage_paths.y6deep.csv
CPU times: user 3.81 s, sys: 1.8 s, total: 5.61 s
Wall time: 7.25 s


In [26]:
# Show first 10 lines of DataFrame...
if do_query_decade_filepaths:
    print df0_decade_deep.head(10)

   EXPNUM  CCDNUM BAND    RACMIN     RACMAX    DECCMIN    DECCMAX                                                                                                                                              ILINK                                                                                                                                       CLINK
0  161052       1    g  8.707232   9.119224 -43.302602 -43.151698  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121213-r4157/D00161052/p01/red/immask/D00161052_g_c01_r4157p01_immasked.fits.fz  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121213-r4157/D00161052/p01/cat/D00161052_g_c01_r4157p01_red-fullcat.fits
1  161052       2    g  9.133448   9.544796 -43.303619 -43.153579  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121213-r4157/D00161052/p01/red/immask/D00161052_g_c02_r4157p01_immasked.fits.fz  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121213-r4157/D

### 5.b.  Image sky coordinates

**We will first work with the DESSCI db, and then we will repeat for the DECADE db.**

**DESSCI db:**

In [27]:
%%time

# Query image info from NCSA DESSCI db...

if do_query_des_img_skypos:
        
    # The query...
    #  See:  https://sites.google.com/view/delve-survey/data?pli=1&authuser=1
    query = \
        """
        select i.expnum, i.ccdnum, i.band, 
                i.racmin, i.racmax, i.deccmin, i.deccmax
        from Y6A2_PROCTAG t, Y6A2_IMAGE i 
        where t.tag='%s' and 
                t.pfw_attempt_id=i.pfw_attempt_id and 
                i.filetype='red_immask'
        order by i.expnum, i.ccdnum
    """ % (tag_eff_des)
    
    print query
    
    # Query NCSA DESSCI db...
    connection=ea.connect('dessci')
    df1_des = connection.query_to_pandas(query)
    connection.close()

    # Output results to CSV file...
    output_file = """DES_%s.expimage_skypos.csv""" % (tag_des)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df1_des.to_csv(output_file, index=False)


        select i.expnum, i.ccdnum, i.band, 
                i.racmin, i.racmax, i.deccmin, i.deccmax
        from Y6A2_PROCTAG t, Y6A2_IMAGE i 
        where t.tag='Y6A1_FINALCUT' and 
                t.pfw_attempt_id=i.pfw_attempt_id and 
                i.filetype='red_immask'
        order by i.expnum, i.ccdnum
    
Connecting to DB ** dessci ** ...
../RawData/DES_Y6A2_FINALCUT.expimage_skypos.csv
CPU times: user 2min 2s, sys: 11.1 s, total: 2min 13s
Wall time: 2min 55s


In [28]:
# Show first 10 lines of DataFrame...
if do_query_des_img_skypos:
    print df1_des.head(10)

   EXPNUM  CCDNUM BAND      RACMIN      RACMAX   DECCMIN   DECCMAX
0  154833       1    g  359.541352  359.840757  0.791482  0.940849
1  154833       2    g  359.851496    0.151182  0.790486  0.940536
2  154833       3    g    0.162466    0.461681  0.790377  0.940262
3  154833       4    g  359.385804  359.685217  0.627556  0.776864
4  154833       5    g  359.696217  359.995803  0.627553  0.776854
5  154833       6    g    0.006904    0.306416  0.626793  0.776701
6  154833       7    g    0.317603    0.616586  0.625907  0.775974
7  154833       8    g  359.230485  359.529620  0.463187  0.612767
8  154833       9    g  359.540415  359.840200  0.463296  0.612928
9  154833      10    g  359.851175    0.150962  0.462909  0.612698


In [29]:
# Perform inner join with Deep Field list.

if do_query_des_img_skypos:
    
    df1_des_deep = pd.merge(df1_des, df_uniqExposureList, \
                            on=['EXPNUM'], \
                            how='inner', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df1_des_deep if colname.endswith('_y')]
    df1_des_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """DES_%s.expimage_skypos.y6_deep.csv""" % (tag_des)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df1_des_deep.to_csv(output_file, index=False)

../RawData/DES_Y6A2_FINALCUT.expimage_skypos.y6_deep.csv


In [30]:
# Show first 10 lines of DataFrame...
if do_query_des_img_skypos:
    print df1_des_deep.head(10)

   EXPNUM  CCDNUM BAND     RACMIN     RACMAX    DECCMIN    DECCMAX
0  154852       1    z  52.130154  52.467110 -27.270366 -27.120527
1  154852       2    z  52.479072  52.815881 -27.271100 -27.121378
2  154852       3    z  52.828725  53.165481 -27.270409 -27.120895
3  154852       4    z  51.954251  52.291958 -27.434045 -27.283827
4  154852       5    z  52.303990  52.641447 -27.434251 -27.284821
5  154852       6    z  52.654087  52.991499 -27.434309 -27.285113
6  154852       7    z  53.003585  53.341103 -27.434287 -27.284375
7  154852       8    z  51.778092  52.116216 -27.597805 -27.447229
8  154852       9    z  52.127792  52.466222 -27.598525 -27.448316
9  154852      10    z  52.478494  52.816724 -27.598652 -27.449132


**DECADE db:**

In [31]:
%%time

# Query image info from NCSA DECADE db...

if do_query_decade_img_skypos:

    print 'Query image info from NCSA DECADE db...'

    # The query...
    #  See:  https://sites.google.com/view/delve-survey/data?pli=1&authuser=1
    query = \
        """
        select i.expnum, i.ccdnum, i.band, 
                i.racmin, i.racmax, i.deccmin, i.deccmax
        from decade.proctag t, decade.image i 
        where t.tag='%s' and 
                t.pfw_attempt_id=i.pfw_attempt_id and 
                i.filetype='red_immask'
        order by i.expnum, i.ccdnum
    """ % (tag_decade)
    
    print query
    
    # Query NCSA DECADE db...
    connection=ea.connect('delve')
    df1_decade = connection.query_to_pandas(query)
    connection.close()

    # Output results to CSV file...
    output_file = """%s.expimage_skypos.csv""" % (tag_decade)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df1_decade.to_csv(output_file, index=False)

Query image info from NCSA DECADE db...

        select i.expnum, i.ccdnum, i.band, 
                i.racmin, i.racmax, i.deccmin, i.deccmax
        from decade.proctag t, decade.image i 
        where t.tag='DECADE_FINALCUT' and 
                t.pfw_attempt_id=i.pfw_attempt_id and 
                i.filetype='red_immask'
        order by i.expnum, i.ccdnum
    
Connecting to DB ** delve ** ...
../RawData/DECADE_FINALCUT.expimage_skypos.csv
CPU times: user 2min 25s, sys: 14.8 s, total: 2min 39s
Wall time: 7min 2s


In [32]:
# Show first 10 lines of DataFrame...
if do_query_decade_img_skypos:
    print df1_decade.head(10)

   EXPNUM  CCDNUM BAND      RACMIN    RACMAX    DECCMIN    DECCMAX
0  145425       1    g    0.112582  0.454720 -28.873491 -28.723264
1  145425       2    g    0.466757  0.808707 -28.874450 -28.724455
2  145425       3    g    0.821744  1.163469 -28.873764 -28.724304
3  145425       4    g  359.933866  0.276822 -29.037047 -28.886446
4  145425       5    g    0.288915  0.631558 -29.037517 -28.887813
5  145425       6    g    0.644289  0.986784 -29.037763 -28.888370
6  145425       7    g    0.999130  1.341681 -29.037749 -28.887894
7  145425       8    g  359.754835  0.098286 -29.200655 -29.049691
8  145425       9    g    0.109900  0.453584 -29.201669 -29.051173
9  145425      10    g    0.465927  0.809280 -29.202020 -29.052287


In [33]:
# Perform inner join with Deep Field list.

if do_query_decade_img_skypos:
    
    df1_decade_deep = pd.merge(df1_decade, df_uniqExposureList, \
                            on=['EXPNUM'], \
                            how='inner', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df1_decade_deep if colname.endswith('_y')]
    df1_decade_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """%s.expimage_skypos.y6_deep.csv""" % (tag_decade)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df1_decade_deep.to_csv(output_file, index=False)

../RawData/DECADE_FINALCUT.expimage_skypos.y6_deep.csv


In [34]:
# Show first 10 lines of DataFrame...
if do_query_decade_img_skypos:
    print df1_decade_deep.head(10)

   EXPNUM  CCDNUM BAND    RACMIN     RACMAX    DECCMIN    DECCMAX
0  161052       1    g  8.707232   9.119224 -43.302602 -43.151698
1  161052       2    g  9.133448   9.544796 -43.303619 -43.153579
2  161052       3    g  9.560469   9.971903 -43.302922 -43.152866
3  161052       4    g  8.491183   8.904829 -43.465881 -43.314283
4  161052       5    g  8.918917   9.331771 -43.466742 -43.316674
5  161052       6    g  9.347063   9.759700 -43.466786 -43.317509
6  161052       7    g  9.774170  10.187274 -43.466676 -43.315907
7  161052       8    g  8.274273   8.689189 -43.629041 -43.476758
8  161052       9    g  8.702494   9.117278 -43.630781 -43.479601
9  161052      10    g  9.131899   9.546033 -43.631192 -43.481413


### 5.c.  Exposure Information

**We will first work with the DESSCI db, and then we will repeat for the DECADE db.**

**DESSCI db:**

In [35]:
%%time

# Query exposure final_cut QA from NCSA DECADE db...

if do_query_des_exp_finalcut_qa:

    print 'Query exposure final_cut from NCSA DESSCI db...'

    # The query...
    #  See:  https://sites.google.com/view/delve-survey/data?pli=1&authuser=1

    # Y6A2_QA_SUMMARY in the DESSCI db does not contain ellipticity like 
    #  FINALCUT_EVAL in the DECADE db; so, for consistency, we will query
    #  another quantity instead and then set it to -9999 as a signal value.
    query = \
        """
        select e.expnum, e.radeg, e.decdeg, e.band, 
                concat('/DESFiles/desarchive/',a.archive_path) as path, 
                f.PSF_FWHM as FWHM_ASEC, f.skybrightness, f.skytilt as ELLIPTICITY, f.t_eff, 
                e.program 
        from Y6A2_EXPOSURE e, Y6A2_QA_SUMMARY f, Y6A2_PROCTAG t, PROD.PFW_ATTEMPT@desoper a
        where t.pfw_attempt_id=f.pfw_attempt_id and 
                t.pfw_attempt_id=a.id and f.expnum=e.expnum and 
                t.tag = '%s'
        """ % (tag_eff_des)
    
    print query
 
    # Query NCSA DESSCI db...
    connection=ea.connect('dessci')
    df2_des = connection.query_to_pandas(query)
    connection.close()

    # Signal value for missing ellipticity in Y6A2_QA_SUMMARY...
    df2_des['ELLIPTICITY'] = -9999.
    
    # Output results to CSV file...
    output_file = """DES_%s.exp_finalcut_qa.csv""" % (tag_des)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df2_des.to_csv(output_file, index=False)

Query exposure final_cut from NCSA DESSCI db...

        select e.expnum, e.radeg, e.decdeg, e.band, 
                concat('/DESFiles/desarchive/',a.archive_path) as path, 
                f.PSF_FWHM as FWHM_ASEC, f.skybrightness, f.skytilt as ELLIPTICITY, f.t_eff, 
                e.program 
        from Y6A2_EXPOSURE e, Y6A2_QA_SUMMARY f, Y6A2_PROCTAG t, PROD.PFW_ATTEMPT@desoper a
        where t.pfw_attempt_id=f.pfw_attempt_id and 
                t.pfw_attempt_id=a.id and f.expnum=e.expnum and 
                t.tag = 'Y6A1_FINALCUT'
        
Connecting to DB ** dessci ** ...
../RawData/DES_Y6A2_FINALCUT.exp_finalcut_qa.csv
CPU times: user 3.09 s, sys: 229 ms, total: 3.32 s
Wall time: 5.57 s


In [36]:
# Show first 10 lines of DataFrame...
if do_query_des_exp_finalcut_qa:
    print df2_des.head(10)

   EXPNUM       RADEG     DECDEG BAND                                                                 PATH  FWHM_ASEC  SKYBRIGHTNESS  ELLIPTICITY     T_EFF           PROGRAM
0  722909   49.999996   0.000000    g  /DESFiles/desarchive/OPS/finalcut/Y5A1/r3519/20180218/D00722909/p01      1.107           9.57      -9999.0  0.073423  photom-std-field
1  716595   79.705517 -23.310300    Y  /DESFiles/desarchive/OPS/finalcut/Y5A1/r3519/20180127/D00716595/p01      0.689          23.23      -9999.0  0.861341            survey
2  704645  143.250000  -0.083333    r  /DESFiles/desarchive/OPS/finalcut/Y5A1/r3518/20171212/D00704645/p01      0.938          14.15      -9999.0  0.033785  photom-std-field
3  713153   63.824700 -48.672300    r  /DESFiles/desarchive/OPS/finalcut/Y5A1/r3519/20180117/D00713153/p01      0.898           2.74      -9999.0  0.968459            survey
4  718952   65.142796 -29.397200    r  /DESFiles/desarchive/OPS/finalcut/Y5A1/r3519/20180207/D00718952/p01      0.872           2.

In [37]:
# Perform inner join with Deep Field list.

if do_query_des_exp_finalcut_qa:
    
    df2_des_deep = pd.merge(df2_des, df_uniqExposureList, \
                            on=['EXPNUM'], \
                            how='inner', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df2_des_deep if colname.endswith('_y')]
    df2_des_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """DES_%s.exp_finalcut_qa.y6_deep.csv""" % (tag_des)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df2_des_deep.to_csv(output_file, index=False)

../RawData/DES_Y6A2_FINALCUT.exp_finalcut_qa.y6_deep.csv


In [38]:
# Show first 10 lines of DataFrame...
if do_query_des_exp_finalcut_qa:
    print df2_des_deep.head(10)

   EXPNUM      RADEG     DECDEG BAND                                                                 PATH  FWHM_ASEC  SKYBRIGHTNESS  ELLIPTICITY     T_EFF    PROGRAM
0  670024  41.193929  -0.989706    z  /DESFiles/desarchive/OPS/finalcut/Y5A1/r3515/20170820/D00670024/p01      1.227      21.340000      -9999.0  0.363887  supernova
1  671688  36.451200  -4.598750    z  /DESFiles/desarchive/OPS/finalcut/Y5A1/r3515/20170824/D00671688/p01      1.142      17.639999      -9999.0  0.504982  supernova
2  693605  34.472308  -4.931639    z  /DESFiles/desarchive/OPS/finalcut/Y5A1/r3517/20171106/D00693605/p01      0.865      23.900000      -9999.0  0.635940  supernova
3  678878  52.650121 -28.100778    g  /DESFiles/desarchive/OPS/finalcut/Y5A1/r3515/20170917/D00678878/p01      1.057       0.870000      -9999.0  0.984373  supernova
4  508823  52.648400 -28.100000    z  /DESFiles/desarchive/OPS/finalcut/Y5A1/r3525/20160106/D00508823/p01      1.178      17.940001      -9999.0  0.468065  supernova
5  5

**DECADE db:**

In [39]:
%%time

# Query exposure final_cut QA from NCSA DECADE db...

if do_query_decade_exp_finalcut_qa:

    print 'Query exposure final_cut from NCSA DECADE db...'

    # The query...
    #  See:  https://sites.google.com/view/delve-survey/data?pli=1&authuser=1

    query = \
        """
        select e.expnum, e.radeg, e.decdeg, e.band, 
                concat('/deca_archive/',a.archive_path) as path, 
                f.fwhm_asec, f.skybrightness, f.ellipticity, f.t_eff, 
                e.program 
        from exposure e, finalcut_eval f, proctag t, pfw_attempt a 
        where t.pfw_attempt_id=f.pfw_attempt_id and 
                t.pfw_attempt_id=a.id and f.expnum=e.expnum and 
                t.tag = '%s'
        """ % (tag_decade)
    
    print query
 
    # Query NCSA DECADE db...
    connection=ea.connect('delve')
    df2_decade = connection.query_to_pandas(query)
    connection.close()

    # Output results to CSV file...
    output_file = """%s.exp_finalcut_qa.csv""" % (tag_decade)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df2_decade.to_csv(output_file, index=False)

Query exposure final_cut from NCSA DECADE db...

        select e.expnum, e.radeg, e.decdeg, e.band, 
                concat('/deca_archive/',a.archive_path) as path, 
                f.fwhm_asec, f.skybrightness, f.ellipticity, f.t_eff, 
                e.program 
        from exposure e, finalcut_eval f, proctag t, pfw_attempt a 
        where t.pfw_attempt_id=f.pfw_attempt_id and 
                t.pfw_attempt_id=a.id and f.expnum=e.expnum and 
                t.tag = 'DECADE_FINALCUT'
        
Connecting to DB ** delve ** ...
../RawData/DECADE_FINALCUT.exp_finalcut_qa.csv
CPU times: user 4.02 s, sys: 312 ms, total: 4.33 s
Wall time: 6.03 s


In [40]:
# Show first 10 lines of DataFrame...
if do_query_decade_exp_finalcut_qa:
    print df2_decade.head(10)

   EXPNUM       RADEG     DECDEG BAND                                                          PATH  FWHM_ASEC  SKYBRIGHTNESS  ELLIPTICITY  T_EFF                                                                                 PROGRAM
0  182928  159.150000 -27.517778    i  /deca_archive/DEC/finalcut/Y5A1/20130228-r4204/D00182928/p01      0.962          14.05        0.061  0.452  The DECam Advanced LIGO Pilot Survey: A Benchmark for the Era of Joint GW/EM Astronomy
1  182998  235.331400 -12.966082    r  /deca_archive/DEC/finalcut/Y5A1/20130228-r4204/D00182998/p02      1.036          25.77        0.058  0.078  The DECam Advanced LIGO Pilot Survey: A Benchmark for the Era of Joint GW/EM Astronomy
2  182902  154.935417 -33.123333    z  /deca_archive/DEC/finalcut/Y5A1/20130228-r4204/D00182902/p01      0.892          17.83        0.060  0.804  The DECam Advanced LIGO Pilot Survey: A Benchmark for the Era of Joint GW/EM Astronomy
3  182823  154.877083 -35.323333    i  /deca_archive/DEC/finalcu

In [41]:
# Perform inner join with Deep Field list.

if do_query_decade_exp_finalcut_qa:
    
    df2_decade_deep = pd.merge(df2_decade, df_uniqExposureList, \
                            on=['EXPNUM'], \
                            how='inner', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df2_decade_deep if colname.endswith('_y')]
    df2_decade_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """%s.exp_finalcut_qa.y6_deep.csv""" % (tag_decade)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df2_decade_deep.to_csv(output_file, index=False)

../RawData/DECADE_FINALCUT.exp_finalcut_qa.y6_deep.csv


In [42]:
# Show first 10 lines of DataFrame...
if do_query_decade_exp_finalcut_qa:
    print df2_decade_deep.head(10)

   EXPNUM      RADEG     DECDEG BAND                                                          PATH  FWHM_ASEC  SKYBRIGHTNESS  ELLIPTICITY  T_EFF                                                            PROGRAM
0  951313  63.777404 -48.487600    i  /deca_archive/DEC/finalcut/Y5A1/20201102-r5161/D00951313/p01      0.841      14.850000        0.092  0.553                                                               EDFS
1  923727  35.664496  -6.412110    z  /deca_archive/DEC/finalcut/Y5A1/20191230-r5029/D00923727/p02      0.634      22.230000        0.109  1.233  DECam Early Imaging of AGN Reverberation Mapping Fields in SDSS-V
2  898468  52.653283 -28.097250    z  /deca_archive/DEC/finalcut/Y5A1/20190926-r4975/D00898468/p01      1.332      12.890000        0.069  0.513  DECam Early Imaging of AGN Reverberation Mapping Fields in SDSS-V
3  923700  41.193496  -0.985334    g  /deca_archive/DEC/finalcut/Y5A1/20191230-r5029/D00923700/p02      0.885       1.860000        0.201  0.649  DECam 

### 5.d.  Links to Final Cut Catalogs for all Images

**We will first work with the DESSCI db, and then we will repeat for the DECADE db.**

**DESSCI db:**

In [43]:
%%time

# Query image CLINKs from NCSA DESSCI db...

if do_query_des_expimage_clinks:
    
    print 'Query image CLINKs from NCSA DESSCI db...'

    # The query...
    #  See:  https://sites.google.com/view/delve-survey/data?pli=1&authuser=1
    
    query = \
        """
        select c.expnum, c.ccdnum, c.band, 
        'https://desar2.cosmology.illinois.edu/DESFiles/desarchive/'||fai.path||'/'||c.filename||fai.compression as clink
        from Y6A2_PROCTAG t, Y6A2_CATALOG c, Y6A2_FILE_ARCHIVE_INFO fai 
        where t.tag='%s' and t.pfw_attempt_id=c.pfw_attempt_id and 
              c.filetype='cat_finalcut' and c.filename=fai.filename 
        order by expnum, ccdnum
        """ % (tag_eff_des)
    
    print query
    
    # Query NCSA DESSCI db...
    connection=ea.connect('dessci')
    df3_des = connection.query_to_pandas(query)
    connection.close()

    # (Useful for later):  Add 'FILENAME' column to df3_des...
    df3_des['FILENAME'] = df3_des['CLINK'].apply(lambda x: os.path.basename(x))
    
    # Output results to CSV file...
    output_file = """DES_%s.expimage_clinks.csv""" % (tag_des)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df3_des.to_csv(output_file, index=False)

Query image CLINKs from NCSA DESSCI db...

        select c.expnum, c.ccdnum, c.band, 
        'https://desar2.cosmology.illinois.edu/DESFiles/desarchive/'||fai.path||'/'||c.filename||fai.compression as clink
        from Y6A2_PROCTAG t, Y6A2_CATALOG c, Y6A2_FILE_ARCHIVE_INFO fai 
        where t.tag='Y6A1_FINALCUT' and t.pfw_attempt_id=c.pfw_attempt_id and 
              c.filetype='cat_finalcut' and c.filename=fai.filename 
        order by expnum, ccdnum
        
Connecting to DB ** dessci ** ...
../RawData/DES_Y6A2_FINALCUT.expimage_clinks.csv
CPU times: user 1min 59s, sys: 19.5 s, total: 2min 19s
Wall time: 3min 34s


In [44]:
# Show first 10 lines of DataFrame...
if do_query_des_expimage_clinks:
    print df3_des.head(10)

   EXPNUM  CCDNUM BAND                                                                                                                                                  CLINK                                   FILENAME
0  154833       1    g  https://desar2.cosmology.illinois.edu/DESFiles/desarchive/OPS/finalcut/Y5A1/r3727/20121123/D00154833/p02/cat/D00154833_g_c01_r3727p02_red-fullcat....  D00154833_g_c01_r3727p02_red-fullcat.fits
1  154833       2    g  https://desar2.cosmology.illinois.edu/DESFiles/desarchive/OPS/finalcut/Y5A1/r3727/20121123/D00154833/p02/cat/D00154833_g_c02_r3727p02_red-fullcat....  D00154833_g_c02_r3727p02_red-fullcat.fits
2  154833       3    g  https://desar2.cosmology.illinois.edu/DESFiles/desarchive/OPS/finalcut/Y5A1/r3727/20121123/D00154833/p02/cat/D00154833_g_c03_r3727p02_red-fullcat....  D00154833_g_c03_r3727p02_red-fullcat.fits
3  154833       4    g  https://desar2.cosmology.illinois.edu/DESFiles/desarchive/OPS/finalcut/Y5A1/r3727/20121123/D00154833/p02/cat

In [45]:
# Perform inner join with Deep Field list.

if do_query_des_expimage_clinks:
    
    df3_des_deep = pd.merge(df3_des, df_uniqExposureList, \
                            on=['EXPNUM'], \
                            how='inner', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df3_des_deep if colname.endswith('_y')]
    df3_des_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """DES_%s.expimage_clinks.y6_deep.csv""" % (tag_des)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df3_des_deep.to_csv(output_file, index=False)

../RawData/DES_Y6A2_FINALCUT.expimage_clinks.y6_deep.csv


In [46]:
# Show first 10 lines of DataFrame...
if do_query_des_expimage_clinks:
    print df3_des_deep.head(10)

   EXPNUM  CCDNUM BAND                                                                                                                                                  CLINK                                   FILENAME
0  154852       1    z  https://desar2.cosmology.illinois.edu/DESFiles/desarchive/OPS/finalcut/Y5A1/r3727/20121123/D00154852/p02/cat/D00154852_z_c01_r3727p02_red-fullcat....  D00154852_z_c01_r3727p02_red-fullcat.fits
1  154852       2    z  https://desar2.cosmology.illinois.edu/DESFiles/desarchive/OPS/finalcut/Y5A1/r3727/20121123/D00154852/p02/cat/D00154852_z_c02_r3727p02_red-fullcat....  D00154852_z_c02_r3727p02_red-fullcat.fits
2  154852       3    z  https://desar2.cosmology.illinois.edu/DESFiles/desarchive/OPS/finalcut/Y5A1/r3727/20121123/D00154852/p02/cat/D00154852_z_c03_r3727p02_red-fullcat....  D00154852_z_c03_r3727p02_red-fullcat.fits
3  154852       4    z  https://desar2.cosmology.illinois.edu/DESFiles/desarchive/OPS/finalcut/Y5A1/r3727/20121123/D00154852/p02/cat

**DECADE db:**

In [47]:
%%time

# Query image CLINKs from NCSA DECADE db...

if do_query_decade_expimage_clinks:
    
    print 'Query image CLINKs from NCSA DECADE db...'

    # The query...
    #  See:  https://sites.google.com/view/delve-survey/data?pli=1&authuser=1
    
    query = \
        """
        select c.expnum, c.ccdnum, c.band, 
        'https://decade.ncsa.illinois.edu/deca_archive/'||fai.path||'/'||c.filename||fai.compression as clink
        from decade.proctag t, decade.catalog c, decade.file_archive_info fai 
        where t.tag='%s' and t.pfw_attempt_id=c.pfw_attempt_id and 
              c.filetype='cat_finalcut' and c.filename=fai.filename 
        order by expnum, ccdnum
        """ % (tag_decade)
    
    print query
    
    # Query NCSA DECADE db...
    connection=ea.connect('delve')
    df3_decade = connection.query_to_pandas(query)
    connection.close()

    # (Useful for later):  Add 'FILENAME' column to df3_decade...
    df3_decade['FILENAME'] = df3_decade['CLINK'].apply(lambda x: os.path.basename(x))
    
    # Output results to CSV file...
    output_file = """%s.expimage_clinks.csv""" % (tag_decade)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df3_decade.to_csv(output_file, index=False)

Query image CLINKs from NCSA DECADE db...

        select c.expnum, c.ccdnum, c.band, 
        'https://decade.ncsa.illinois.edu/deca_archive/'||fai.path||'/'||c.filename||fai.compression as clink
        from decade.proctag t, decade.catalog c, decade.file_archive_info fai 
        where t.tag='DECADE_FINALCUT' and t.pfw_attempt_id=c.pfw_attempt_id and 
              c.filetype='cat_finalcut' and c.filename=fai.filename 
        order by expnum, ccdnum
        
Connecting to DB ** delve ** ...
../RawData/DECADE_FINALCUT.expimage_clinks.csv
CPU times: user 2min 28s, sys: 25.8 s, total: 2min 54s
Wall time: 6min 49s


In [48]:
# Show first 10 lines of DataFrame...
if do_query_decade_expimage_clinks:
    print df3_decade.head(10)

   EXPNUM  CCDNUM BAND                                                                                                                                       CLINK                                   FILENAME
0  145425       1    g  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121101-r4148/D00145425/p01/cat/D00145425_g_c01_r4148p01_red-fullcat.fits  D00145425_g_c01_r4148p01_red-fullcat.fits
1  145425       2    g  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121101-r4148/D00145425/p01/cat/D00145425_g_c02_r4148p01_red-fullcat.fits  D00145425_g_c02_r4148p01_red-fullcat.fits
2  145425       3    g  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121101-r4148/D00145425/p01/cat/D00145425_g_c03_r4148p01_red-fullcat.fits  D00145425_g_c03_r4148p01_red-fullcat.fits
3  145425       4    g  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121101-r4148/D00145425/p01/cat/D00145425_g_c04_r4148p01_red-fullcat.fits  D00145425_g_

In [49]:
# Perform inner join with Deep Field list.

if do_query_decade_expimage_clinks:
    
    df3_decade_deep = pd.merge(df3_decade, df_uniqExposureList, \
                            on=['EXPNUM'], \
                            how='inner', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df3_decade_deep if colname.endswith('_y')]
    df3_decade_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """%s.expimage_clinks.y6_deep.csv""" % (tag_decade)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df3_decade_deep.to_csv(output_file, index=False)

../RawData/DECADE_FINALCUT.expimage_clinks.y6_deep.csv


In [50]:
# Show first 10 lines of DataFrame...
if do_query_decade_expimage_clinks:
    print df3_decade_deep.head(10)

   EXPNUM  CCDNUM BAND                                                                                                                                       CLINK                                   FILENAME
0  161052       1    g  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121213-r4157/D00161052/p01/cat/D00161052_g_c01_r4157p01_red-fullcat.fits  D00161052_g_c01_r4157p01_red-fullcat.fits
1  161052       2    g  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121213-r4157/D00161052/p01/cat/D00161052_g_c02_r4157p01_red-fullcat.fits  D00161052_g_c02_r4157p01_red-fullcat.fits
2  161052       3    g  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121213-r4157/D00161052/p01/cat/D00161052_g_c03_r4157p01_red-fullcat.fits  D00161052_g_c03_r4157p01_red-fullcat.fits
3  161052       4    g  https://decade.ncsa.illinois.edu/deca_archive/DEC/finalcut/Y5A1/20121213-r4157/D00161052/p01/cat/D00161052_g_c04_r4157p01_red-fullcat.fits  D00161052_g_

### 5.e.  Exposure, qa_summary, and image information

**We will first work with the DESSCI db, and then we will repeat for the DECADE db.**

**DESSCI db**

In [51]:
%%time

# Query exposure/QA/image info from NCSA DESSCI db...

if do_query_des_exp_qa:

    print 'Query exposure/QA/image info from NCSA DESSCI db...'

    # The query...
    #  See:  https://github.com/DouglasLeeTucker/DECam_PGCM/blob/master/bin/rawdata_exp_query.py

    query = \
        """
        select e.expnum, 
               e.radeg as EXPRA, e.decdeg as EXPDEC, e.exptime, e.airmass, 
               e.band, e.nite, e.mjd_obs, 
               REPLACE(e.field,   ',', ' ') as field, 
               REPLACE(e.object,  ',', ' ') as object, 
               REPLACE(e.program, ',', ' ') as program, 
               t.pfw_attempt_id, t.tag, 
               qa.source, qa.t_eff, qa.psf_fwhm, qa.f_eff, 
               qa.skybrightness, qa.b_eff, qa.cloud_apass, qa.cloud_nomad,
               qa.n_apass, qa.n_nomad, qa.c_eff, qa.skytilt, qa.astrom_sigma,
               qa.astrom_offset, qa.astrom_ndets, qa.astrom_chi2, qa.nobjects,
               qa.flag, qa.calnac, qa.cloud_des, qa.n_des
        from Y6A2_EXPOSURE e, Y6A2_PROCTAG t, Y6A2_QA_SUMMARY qa
        where t.pfw_attempt_id=qa.pfw_attempt_id and
              e.expnum=qa.expnum and 
              t.tag='%s' 
        order by qa.expnum
        """ % (tag_eff_des)

    print query
    
    # Query NCSA DESSCI db...
    connection=ea.connect('dessci')
    df4_des = connection.query_to_pandas(query)
    connection.close()

    # Output results to CSV file...
    output_file = """DES_%s.expinfo.csv""" % (tag_des)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df4_des.to_csv(output_file, index=False)

Query exposure/QA/image info from NCSA DESSCI db...

        select e.expnum, 
               e.radeg as EXPRA, e.decdeg as EXPDEC, e.exptime, e.airmass, 
               e.band, e.nite, e.mjd_obs, 
               REPLACE(e.field,   ',', ' ') as field, 
               REPLACE(e.object,  ',', ' ') as object, 
               REPLACE(e.program, ',', ' ') as program, 
               t.pfw_attempt_id, t.tag, 
               qa.source, qa.t_eff, qa.psf_fwhm, qa.f_eff, 
               qa.skybrightness, qa.b_eff, qa.cloud_apass, qa.cloud_nomad,
               qa.n_apass, qa.n_nomad, qa.c_eff, qa.skytilt, qa.astrom_sigma,
               qa.astrom_offset, qa.astrom_ndets, qa.astrom_chi2, qa.nobjects,
               qa.flag, qa.calnac, qa.cloud_des, qa.n_des
        from Y6A2_EXPOSURE e, Y6A2_PROCTAG t, Y6A2_QA_SUMMARY qa
        where t.pfw_attempt_id=qa.pfw_attempt_id and
              e.expnum=qa.expnum and 
              t.tag='Y6A1_FINALCUT' 
        order by qa.expnum
        
Connecting to 

In [52]:
# Show first 10 lines of DataFrame...
if do_query_des_exp_qa:
    print df4_des.head(10)

   EXPNUM  EXPRA  EXPDEC  EXPTIME  AIRMASS BAND      NITE       MJD_OBS FIELD            OBJECT PROGRAM  PFW_ATTEMPT_ID            TAG         SOURCE     T_EFF  PSF_FWHM     F_EFF  SKYBRIGHTNESS  B_EFF  CLOUD_APASS  CLOUD_NOMAD  N_APASS  N_NOMAD  C_EFF   SKYTILT  ASTROM_SIGMA  ASTROM_OFFSET  ASTROM_NDETS   ASTROM_CHI2  NOBJECTS  FLAG  CALNAC  CLOUD_DES   N_DES
0  154833    0.0     0.0     15.0     1.16    g  20121123  56255.015228  None  Stripe82Marriner  DES SV         2322714  Y6A1_FINALCUT  Y6A1_FINALCUT  0.011688     1.121  0.730514      67.699997  0.016        0.022        0.030    591.0   1330.0    1.0  0.016543      0.056932       0.004175         941.0    111.292999  208472.0     1       1      0.031  1857.0
1  154834    0.0     0.0     15.0     1.16    r  20121123  56255.015734  None  Stripe82Marriner  DES SV         2323714  Y6A1_FINALCUT  Y6A1_FINALCUT  0.018568     1.090  0.687719      99.849998  0.027        0.078        0.052    243.0    660.0    1.0  0.020339      0.0497

In [53]:
# Perform inner join with Deep Field list.

if do_query_des_exp_qa:
    
    df4_des_deep = pd.merge(df4_des, df_uniqExposureList, \
                            on=['EXPNUM'], \
                            how='inner', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df4_des_deep if colname.endswith('_y')]
    df4_des_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """DES_%s.expinfo.y6_deep.csv""" % (tag_des)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df4_des_deep.to_csv(output_file, index=False)

../RawData/DES_Y6A2_FINALCUT.expinfo.y6_deep.csv


In [54]:
# Show first 10 lines of DataFrame...
if do_query_des_exp_qa:
    print df4_des_deep.head(10)

   EXPNUM      EXPRA  EXPDEC  EXPTIME  AIRMASS BAND      NITE       MJD_OBS  FIELD                             OBJECT    PROGRAM  PFW_ATTEMPT_ID            TAG         SOURCE     T_EFF  PSF_FWHM     F_EFF  SKYBRIGHTNESS  B_EFF  CLOUD_APASS  CLOUD_NOMAD  N_APASS  N_NOMAD  C_EFF   SKYTILT  ASTROM_SIGMA  ASTROM_OFFSET  ASTROM_NDETS  ASTROM_CHI2  NOBJECTS  FLAG  CALNAC  CLOUD_DES    N_DES
0  154852  52.648333 -28.100    330.0     1.35    z  20121123  56255.033040  SN-C3  DES supernova hex SN-C3 tiling 76  supernova         2320793  Y6A1_FINALCUT  Y6A1_FINALCUT  0.302172     1.162  0.521885      28.540001  0.579          NaN        0.130      NaN    676.0    1.0  0.011746      0.035994       0.002818        3711.0    52.912998  117498.0     0       0      0.045  10201.0
1  154854  52.648333 -28.100    330.0     1.30    z  20121123  56255.041350  SN-C3  DES supernova hex SN-C3 tiling 78  supernova         2320795  Y6A1_FINALCUT  Y6A1_FINALCUT  0.360741     1.081  0.600235      27.469999  0.6

In [55]:
#rawdata_des_exp_split

In [56]:
%%time

# Query image info from NCSA DESSCI db...

if do_query_des_img_info:

    print 'Query image info from NCSA DESSCI db...'

    # The query...
    #  See:  https://github.com/DouglasLeeTucker/DECam_PGCM/blob/master/bin/rawdata_img_query.py
    
    #filetype = 'red_starflat'
    filetype = 'red_immask'
    query = \
        """
        with x as (
            select /*+materialize */ t.pfw_attempt_id, qa.expnum 
                from Y6A2_PROCTAG t, Y6A2_QA_SUMMARY qa 
                where t.pfw_attempt_id=qa.pfw_attempt_id and 
                      tag='%s'
                  ) 
        select i.pfw_attempt_id, i.band, i.expnum, i.ccdnum, 
               i.rac1, i.decc1, i.rac2, i.decc2, i.rac3, i.decc3, i.rac4, i.decc4, 
               i.ra_cent, i.dec_cent, i.racmax, i.racmin, i.crossra0 
        from Y6A2_IMAGE i, x 
        where x.pfw_attempt_id=i.pfw_attempt_id and 
              i.filetype='%s' and 
              i.expnum=x.expnum 
        order by i.expnum, i.ccdnum
        """ % (tag_eff_des, filetype)

    print query
    
    # Query NCSA DESSCI db...
    connection=ea.connect('dessci')
    df5_des = connection.query_to_pandas(query)
    connection.close()

    # Output results to CSV file...
    output_file = """DES_%s.imginfo.csv""" % (tag_des)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df5_des.to_csv(output_file, index=False)

Query image info from NCSA DESSCI db...

        with x as (
            select /*+materialize */ t.pfw_attempt_id, qa.expnum 
                from Y6A2_PROCTAG t, Y6A2_QA_SUMMARY qa 
                where t.pfw_attempt_id=qa.pfw_attempt_id and 
                      tag='Y6A1_FINALCUT'
                  ) 
        select i.pfw_attempt_id, i.band, i.expnum, i.ccdnum, 
               i.rac1, i.decc1, i.rac2, i.decc2, i.rac3, i.decc3, i.rac4, i.decc4, 
               i.ra_cent, i.dec_cent, i.racmax, i.racmin, i.crossra0 
        from Y6A2_IMAGE i, x 
        where x.pfw_attempt_id=i.pfw_attempt_id and 
              i.filetype='red_immask' and 
              i.expnum=x.expnum 
        order by i.expnum, i.ccdnum
        
Connecting to DB ** dessci ** ...
../RawData/DES_Y6A2_FINALCUT.imginfo.csv
CPU times: user 4min 57s, sys: 36.9 s, total: 5min 34s
Wall time: 11min 55s


In [57]:
# Show first 10 lines of DataFrame...
if do_query_des_img_info:
    print df5_des.head(10)

   PFW_ATTEMPT_ID BAND  EXPNUM  CCDNUM        RAC1     DECC1        RAC2     DECC2        RAC3     DECC3        RAC4     DECC4     RA_CENT  DEC_CENT      RACMAX      RACMIN CROSSRA0
0         2322714    g  154833       1  359.541671  0.940849  359.541352  0.791562  359.840479  0.791482  359.840757  0.940604  359.691015  0.866162  359.840757  359.541352        N
1         2322714    g  154833       2  359.852037  0.940536  359.851496  0.791422    0.150789  0.790486    0.151182  0.939595    0.001342  0.865582    0.151182  359.851496        Y
2         2322714    g  154833       3    0.162616  0.940262    0.162466  0.791153    0.461572  0.790377    0.461681  0.939647    0.312065  0.865398    0.461681    0.162466        N
3         2322714    g  154833       4  359.386223  0.776864  359.385804  0.627556  359.684813  0.627583  359.685217  0.776797  359.535437  0.702248  359.685217  359.385804        N
4         2322714    g  154833       5  359.696546  0.776854  359.696217  0.627656  359.99

In [58]:
# Perform inner join with Deep Field list.

if do_query_des_img_info:

    df5_des_deep = pd.merge(df5_des, df_uniqExposureList, \
                            on=['EXPNUM'], \
                            how='inner', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df5_des_deep if colname.endswith('_y')]
    df5_des_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """DES_%s.imginfo.y6_deep.csv""" % (tag_des)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df5_des_deep.to_csv(output_file, index=False)

../RawData/DES_Y6A2_FINALCUT.imginfo.y6_deep.csv


In [59]:
# Show first 10 lines of DataFrame...
if do_query_des_img_info:
    print df5_des_deep.head(10)

   PFW_ATTEMPT_ID BAND  EXPNUM  CCDNUM       RAC1      DECC1       RAC2      DECC2       RAC3      DECC3       RAC4      DECC4    RA_CENT   DEC_CENT     RACMAX     RACMIN CROSSRA0
0         2320793    z  154852       1  52.131127 -27.120527  52.130154 -27.269728  52.466676 -27.270366  52.467110 -27.121305  52.298702 -27.195535  52.467110  52.130154        N
1         2320793    z  154852       2  52.479785 -27.121378  52.479072 -27.270434  52.815817 -27.271100  52.815881 -27.122048  52.647600 -27.196260  52.815881  52.479072        N
2         2320793    z  154852       3  52.828725 -27.121356  52.828955 -27.270409  53.165481 -27.270088  53.164714 -27.120895  52.996952 -27.195740  53.165481  52.828725        N
3         2320793    z  154852       4  51.955577 -27.283827  51.954251 -27.433078  52.291139 -27.434045  52.291958 -27.284860  52.123132 -27.358997  52.291958  51.954251        N
4         2320793    z  154852       5  52.304707 -27.284821  52.303990 -27.433994  52.641400 -27.43

**DECADE db**

In [60]:
%%time

# Query exposure/QA/image info from NCSA DECADE db...

if do_query_decade_exp_qa:

    print 'Query exposure/QA/image info from NCSA DECADE db...'

    # The query...
    #  See:  https://github.com/DouglasLeeTucker/DECam_PGCM/blob/master/bin/rawdata_exp_query.py

    query = \
        """
        select e.expnum, 
               e.radeg as EXPRA, e.decdeg as EXPDEC, e.exptime, e.airmass, 
               e.band, e.nite, e.mjd_obs, 
               REPLACE(e.field,   ',', ' ') as field, 
               REPLACE(e.object,  ',', ' ') as object, 
               REPLACE(e.program, ',', ' ') as program, 
               t.pfw_attempt_id, t.tag, 
               qa.source, qa.t_eff, qa.psf_fwhm, qa.f_eff, 
               qa.skybrightness, qa.b_eff, qa.cloud_apass, qa.cloud_nomad,
               qa.n_apass, qa.n_nomad, qa.c_eff, qa.skytilt, qa.astrom_sigma,
               qa.astrom_offset, qa.astrom_ndets, qa.astrom_chi2, qa.nobjects,
               qa.flag, qa.calnac, qa.cloud_des, qa.n_des
        from decade.exposure e, decade.proctag t, decade.qa_summary qa
        where t.pfw_attempt_id=qa.pfw_attempt_id and
              e.expnum=qa.expnum and 
              t.tag='%s' 
        order by qa.expnum
        """ % (tag_decade)

    print query
    
    # Query NCSA DECADE db...
    connection=ea.connect('delve')
    df4_decade = connection.query_to_pandas(query)
    connection.close()

    # Output results to CSV file...
    output_file = """%s.expinfo.csv""" % (tag_decade)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df4_decade.to_csv(output_file, index=False)

Query exposure/QA/image info from NCSA DECADE db...

        select e.expnum, 
               e.radeg as EXPRA, e.decdeg as EXPDEC, e.exptime, e.airmass, 
               e.band, e.nite, e.mjd_obs, 
               REPLACE(e.field,   ',', ' ') as field, 
               REPLACE(e.object,  ',', ' ') as object, 
               REPLACE(e.program, ',', ' ') as program, 
               t.pfw_attempt_id, t.tag, 
               qa.source, qa.t_eff, qa.psf_fwhm, qa.f_eff, 
               qa.skybrightness, qa.b_eff, qa.cloud_apass, qa.cloud_nomad,
               qa.n_apass, qa.n_nomad, qa.c_eff, qa.skytilt, qa.astrom_sigma,
               qa.astrom_offset, qa.astrom_ndets, qa.astrom_chi2, qa.nobjects,
               qa.flag, qa.calnac, qa.cloud_des, qa.n_des
        from decade.exposure e, decade.proctag t, decade.qa_summary qa
        where t.pfw_attempt_id=qa.pfw_attempt_id and
              e.expnum=qa.expnum and 
              t.tag='DECADE_FINALCUT' 
        order by qa.expnum
        
Connec

In [61]:
# Show first 10 lines of DataFrame...
if do_query_decade_exp_qa:
    print df4_decade.head(10)

   EXPNUM      EXPRA     EXPDEC  EXPTIME  AIRMASS BAND      NITE       MJD_OBS FIELD          OBJECT                                                                                          PROGRAM  PFW_ATTEMPT_ID              TAG           SOURCE     T_EFF  PSF_FWHM     F_EFF  SKYBRIGHTNESS  B_EFF  CLOUD_APASS  CLOUD_NOMAD  N_APASS  N_NOMAD  C_EFF   SKYTILT  ASTROM_SIGMA  ASTROM_OFFSET  ASTROM_NDETS  ASTROM_CHI2  NOBJECTS  FLAG  CALNAC  CLOUD_DES  N_DES
0  145425   0.582917 -29.720417      1.0     1.24    g  20121101  56233.200046  None            JUNK  Synoptic grizY Observations Centered on the Young  high-Galactic Latitude Open Cluster Blanco 1        20925771  DECADE_FINALCUT  DECADE_FINALCUT  0.082259     1.004  0.903948      11.520000  0.091        0.041       -0.001      993     1210    1.0  0.011635      0.094386       0.003409          1183    23.712000     17198     1       1        NaN     -1
1  145427   0.582917 -29.720417      7.0     1.25    g  20121101  56233.201030  No

In [62]:
# Perform inner join with Deep Field list.

if do_query_decade_exp_qa:
    
    df4_decade_deep = pd.merge(df4_decade, df_uniqExposureList, \
                            on=['EXPNUM'], \
                            how='inner', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df4_decade_deep if colname.endswith('_y')]
    df4_decade_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """%s.expinfo.y6_deep.csv""" % (tag_decade)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df4_decade_deep.to_csv(output_file, index=False)

../RawData/DECADE_FINALCUT.expinfo.y6_deep.csv


In [63]:
# Show first 10 lines of DataFrame...
if do_query_decade_exp_qa:
    print df4_decade_deep.head(10)

   EXPNUM     EXPRA     EXPDEC  EXPTIME  AIRMASS BAND      NITE       MJD_OBS FIELD OBJECT PROGRAM  PFW_ATTEMPT_ID              TAG           SOURCE     T_EFF  PSF_FWHM     F_EFF  SKYBRIGHTNESS  B_EFF  CLOUD_APASS  CLOUD_NOMAD  N_APASS  N_NOMAD  C_EFF   SKYTILT  ASTROM_SIGMA  ASTROM_OFFSET  ASTROM_NDETS  ASTROM_CHI2  NOBJECTS  FLAG  CALNAC  CLOUD_DES  N_DES
0  161052  9.352875 -44.129721    120.0     1.42    g  20121213  56275.145544  None    ES1    None        20940893  DECADE_FINALCUT  DECADE_FINALCUT  0.608711     1.139  0.708628           1.22  0.859       -0.002        0.099      418     1291    1.0  0.005376      0.048820       0.007071          2723    35.714001    129158     0       0     -0.013   9046
1  161053  9.352875 -44.129721    120.0     1.43    g  20121213  56275.147255  None    ES1    None        20939897  DECADE_FINALCUT  DECADE_FINALCUT  0.579884     1.164  0.679817           1.23  0.853        0.003        0.109      419     1261    1.0  0.005454      0.049698     

In [64]:
#rawdata_decade_exp_split

In [65]:
%%time

# Query image info from NCSA DECADE db...

if do_query_decade_img_info:

    print 'Query image info from NCSA DECADE db...'

    # The query...
    #  See:  https://github.com/DouglasLeeTucker/DECam_PGCM/blob/master/bin/rawdata_img_query.py

    #filetype = 'red_starflat'
    filetype = 'red_immask'
    query = \
        """
        with x as (
            select /*+materialize */ t.pfw_attempt_id, qa.expnum 
                from decade.proctag t, decade.qa_summary qa 
                where t.pfw_attempt_id=qa.pfw_attempt_id and 
                      tag='%s'
                  ) 
        select i.pfw_attempt_id, i.band, i.expnum, i.ccdnum, 
               i.rac1, i.decc1, i.rac2, i.decc2, i.rac3, i.decc3, i.rac4, i.decc4, 
               i.ra_cent, i.dec_cent, i.racmax, i.racmin, i.crossra0 
        from decade.image i, x 
        where x.pfw_attempt_id=i.pfw_attempt_id and 
              i.filetype='%s' and 
              i.expnum=x.expnum 
        order by i.expnum, i.ccdnum
        """ % (tag_decade, filetype)

    print query
    
    # Query NCSA DECADE db...
    connection=ea.connect('delve')
    df5_decade = connection.query_to_pandas(query)
    connection.close()

    # Output results to CSV file...
    output_file = """%s.imginfo.csv""" % (tag_decade)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df5_decade.to_csv(output_file, index=False)

Query image info from NCSA DECADE db...

        with x as (
            select /*+materialize */ t.pfw_attempt_id, qa.expnum 
                from decade.proctag t, decade.qa_summary qa 
                where t.pfw_attempt_id=qa.pfw_attempt_id and 
                      tag='DECADE_FINALCUT'
                  ) 
        select i.pfw_attempt_id, i.band, i.expnum, i.ccdnum, 
               i.rac1, i.decc1, i.rac2, i.decc2, i.rac3, i.decc3, i.rac4, i.decc4, 
               i.ra_cent, i.dec_cent, i.racmax, i.racmin, i.crossra0 
        from decade.image i, x 
        where x.pfw_attempt_id=i.pfw_attempt_id and 
              i.filetype='red_immask' and 
              i.expnum=x.expnum 
        order by i.expnum, i.ccdnum
        
Connecting to DB ** delve ** ...
../RawData/DECADE_FINALCUT.imginfo.csv
CPU times: user 6min 9s, sys: 50.6 s, total: 7min
Wall time: 12min 17s


In [66]:
# Show first 10 lines of DataFrame...
if do_query_decade_img_info:
    print df5_decade.head(10)

   PFW_ATTEMPT_ID BAND  EXPNUM  CCDNUM        RAC1      DECC1        RAC2      DECC2      RAC3      DECC3      RAC4      DECC4     RA_CENT   DEC_CENT    RACMAX      RACMIN CROSSRA0
0        20925771    g  145425       1    0.113676 -28.723264    0.112582 -28.872546  0.454177 -28.873491  0.454720 -28.724372    0.283732 -28.798490  0.454720    0.112582        N
1        20925771    g  145425       2    0.467584 -28.724455    0.466757 -28.873566  0.808551 -28.874450  0.808707 -28.725343    0.637860 -28.799492  0.808707    0.466757        N
2        20925771    g  145425       3    0.821744 -28.724657    0.821885 -28.873764  1.163469 -28.873570  1.162781 -28.724304    0.992447 -28.799146  1.163469    0.821744        N
3        20925771    g  145425       4  359.935338 -28.886446  359.933866 -29.035748  0.275871 -29.037047  0.276822 -28.887837    0.105384 -28.961832  0.276822  359.933866        Y
4        20925771    g  145425       5    0.289762 -28.887813    0.288915 -29.037007  0.631412 

In [67]:
# Perform inner join with Deep Field list.

if do_query_decade_img_info:

    df5_decade_deep = pd.merge(df5_decade, df_uniqExposureList, \
                            on=['EXPNUM'], \
                            how='inner', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df5_decade_deep if colname.endswith('_y')]
    df5_decade_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """%s.imginfo.y6_deep.csv""" % (tag_decade)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df5_decade_deep.to_csv(output_file, index=False)

../RawData/DECADE_FINALCUT.imginfo.y6_deep.csv


In [68]:
# Show first 10 lines of DataFrame...
if do_query_decade_img_info:
    print df5_decade_deep.head(10)

   PFW_ATTEMPT_ID BAND  EXPNUM  CCDNUM      RAC1      DECC1      RAC2      DECC2       RAC3      DECC3       RAC4      DECC4   RA_CENT   DEC_CENT     RACMAX    RACMIN CROSSRA0
0        20940893    g  161052       1  8.709225 -43.151698  8.707232 -43.300977   9.118308 -43.302602   9.119224 -43.153482  8.913427 -43.227339   9.119224  8.707232        N
1        20940893    g  161052       2  9.134688 -43.153579  9.133448 -43.302691   9.544778 -43.303619   9.544796 -43.154509  9.339379 -43.228715   9.544796  9.133448        N
2        20940893    g  161052       3  9.560469 -43.153812  9.560824 -43.302922   9.971903 -43.302133   9.970473 -43.152866  9.765891 -43.228082   9.971903  9.560469        N
3        20940893    g  161052       4  8.493856 -43.314283  8.491183 -43.463578   8.903202 -43.465881   8.904829 -43.316672  8.698157 -43.390243   8.904829  8.491183        N
4        20940893    g  161052       5  8.920403 -43.316674  8.918917 -43.465867   9.331549 -43.466742   9.331771 -43.31

### 5.f. Inner Joins

**We will first work with the DESSCI db, and then we will repeat for the DECADE db.**

**DESSCI db:**

In [69]:
# Perform inner join between df3_des_deep (CLINK table) and df5_des_deep (image info table).

if do_inner_des_joins:
    
    print 'Perform inner join between df3_des_deep (CLINK table) and df5_des_deep (image info table).'
    
    df35_des_deep = pd.merge(df3_des_deep, df5_des_deep, \
                            on=['EXPNUM', 'CCDNUM'], \
                            how='inner', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df35_des_deep if colname.endswith('_y')]
    to_drop.append('CLINK')
    df35_des_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """DES_%s.expimginfo.y6_deep.csv""" % (tag_des)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df35_des_deep.to_csv(output_file, index=False)

Perform inner join between df3_des_deep (CLINK table) and df5_des_deep (image info table).
../RawData/DES_Y6A2_FINALCUT.expimginfo.y6_deep.csv


In [70]:
# Show first 10 lines of DataFrame...
if do_inner_des_joins:
    print df35_des_deep.head(10)

   EXPNUM  CCDNUM BAND                                   FILENAME  PFW_ATTEMPT_ID       RAC1      DECC1       RAC2      DECC2       RAC3      DECC3       RAC4      DECC4    RA_CENT   DEC_CENT     RACMAX     RACMIN CROSSRA0
0  154852       1    z  D00154852_z_c01_r3727p02_red-fullcat.fits         2320793  52.131127 -27.120527  52.130154 -27.269728  52.466676 -27.270366  52.467110 -27.121305  52.298702 -27.195535  52.467110  52.130154        N
1  154852       2    z  D00154852_z_c02_r3727p02_red-fullcat.fits         2320793  52.479785 -27.121378  52.479072 -27.270434  52.815817 -27.271100  52.815881 -27.122048  52.647600 -27.196260  52.815881  52.479072        N
2  154852       3    z  D00154852_z_c03_r3727p02_red-fullcat.fits         2320793  52.828725 -27.121356  52.828955 -27.270409  53.165481 -27.270088  53.164714 -27.120895  52.996952 -27.195740  53.165481  52.828725        N
3  154852       4    z  D00154852_z_c04_r3727p02_red-fullcat.fits         2320793  51.955577 -27.283827  51.

In [71]:
# Perform inner join between df4_des_deep (exp/qa table) and df35_des_deep (CLINK/image info table).

if do_inner_des_joins:
    
    print 'Perform inner join between df4_des_deep (exp/qa table) and df35_des_deep (CLINK/image info table)...'
    
    df345_des_deep = pd.merge(df4_des_deep, df35_des_deep, \
                            on=['EXPNUM'], \
                            how='right', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df345_des_deep if colname.endswith('_y')]
    df345_des_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """DES_%s.expqaimginfo.y6_deep.csv""" % (tag_des)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df345_des_deep.to_csv(output_file, index=False)

Perform inner join between df4_des_deep (exp/qa table) and df35_des_deep (CLINK/image info table)...
../RawData/DES_Y6A2_FINALCUT.expqaimginfo.y6_deep.csv


In [72]:
# Show first 10 lines of DataFrame...
if do_inner_des_joins:
    print df345_des_deep.head(10)

   EXPNUM      EXPRA  EXPDEC  EXPTIME  AIRMASS BAND      NITE      MJD_OBS  FIELD                             OBJECT    PROGRAM  PFW_ATTEMPT_ID            TAG         SOURCE     T_EFF  PSF_FWHM     F_EFF  SKYBRIGHTNESS  B_EFF  CLOUD_APASS  CLOUD_NOMAD  N_APASS  N_NOMAD  C_EFF   SKYTILT  ASTROM_SIGMA  ASTROM_OFFSET  ASTROM_NDETS  ASTROM_CHI2  NOBJECTS  FLAG  CALNAC  CLOUD_DES    N_DES  CCDNUM                                   FILENAME       RAC1      DECC1       RAC2      DECC2       RAC3      DECC3       RAC4      DECC4    RA_CENT   DEC_CENT     RACMAX     RACMIN CROSSRA0
0  154852  52.648333   -28.1    330.0     1.35    z  20121123  56255.03304  SN-C3  DES supernova hex SN-C3 tiling 76  supernova         2320793  Y6A1_FINALCUT  Y6A1_FINALCUT  0.302172     1.162  0.521885      28.540001  0.579          NaN         0.13      NaN    676.0    1.0  0.011746      0.035994       0.002818        3711.0    52.912998  117498.0     0       0      0.045  10201.0       1  D00154852_z_c01_r3727p02_

**DECADE db:**

In [73]:
# Perform inner join between df3_decade_deep (CLINK table) and df5_decade_deep (image info table).

if do_inner_decade_joins:
    
    print 'Perform inner join between df3_decade_deep (CLINK table) and df5_decade_deep (image info table).'
    
    df35_decade_deep = pd.merge(df3_decade_deep, df5_decade_deep, \
                            on=['EXPNUM', 'CCDNUM'], \
                            how='inner', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df35_decade_deep if colname.endswith('_y')]
    to_drop.append('CLINK')
    df35_decade_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """%s.expimginfo.y6_deep.csv""" % (tag_decade)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df35_decade_deep.to_csv(output_file, index=False)

Perform inner join between df3_decade_deep (CLINK table) and df5_decade_deep (image info table).
../RawData/DECADE_FINALCUT.expimginfo.y6_deep.csv


In [74]:
# Show first 10 lines of DataFrame...
if do_inner_decade_joins:
    print df35_decade_deep.head(10)

   EXPNUM  CCDNUM BAND                                   FILENAME  PFW_ATTEMPT_ID      RAC1      DECC1      RAC2      DECC2       RAC3      DECC3       RAC4      DECC4   RA_CENT   DEC_CENT     RACMAX    RACMIN CROSSRA0
0  161052       1    g  D00161052_g_c01_r4157p01_red-fullcat.fits        20940893  8.709225 -43.151698  8.707232 -43.300977   9.118308 -43.302602   9.119224 -43.153482  8.913427 -43.227339   9.119224  8.707232        N
1  161052       2    g  D00161052_g_c02_r4157p01_red-fullcat.fits        20940893  9.134688 -43.153579  9.133448 -43.302691   9.544778 -43.303619   9.544796 -43.154509  9.339379 -43.228715   9.544796  9.133448        N
2  161052       3    g  D00161052_g_c03_r4157p01_red-fullcat.fits        20940893  9.560469 -43.153812  9.560824 -43.302922   9.971903 -43.302133   9.970473 -43.152866  9.765891 -43.228082   9.971903  9.560469        N
3  161052       4    g  D00161052_g_c04_r4157p01_red-fullcat.fits        20940893  8.493856 -43.314283  8.491183 -43.463578 

In [75]:
# Perform inner join between df4_decade_deep (exp/qa table) and df35_decade_deep (CLINK/image info table).

if do_inner_decade_joins:
    
    print 'Perform inner join between df4_decade_deep (exp/qa table) and df35_decade_deep (CLINK/image info table)...'
    
    df345_decade_deep = pd.merge(df4_decade_deep, df35_decade_deep, \
                            on=['EXPNUM'], \
                            how='right', \
                            suffixes=('','_y')).reset_index(drop=True)

    # Remove any redundant columns...
    to_drop = [colname for colname in df345_decade_deep if colname.endswith('_y')]
    df345_decade_deep.drop(to_drop, axis=1, inplace=True)

    # Output results to CSV file...
    output_file = """%s.expqaimginfo.y6_deep.csv""" % (tag_decade)
    output_file = os.path.join(output_dir, output_file)
    print output_file
    df345_decade_deep.to_csv(output_file, index=False)

Perform inner join between df4_decade_deep (exp/qa table) and df35_decade_deep (CLINK/image info table)...
../RawData/DECADE_FINALCUT.expqaimginfo.y6_deep.csv


In [76]:
# Show first 10 lines of DataFrame...
if do_inner_decade_joins:
    print df345_decade_deep.head(10)

   EXPNUM     EXPRA     EXPDEC  EXPTIME  AIRMASS BAND      NITE       MJD_OBS FIELD OBJECT PROGRAM  PFW_ATTEMPT_ID              TAG           SOURCE     T_EFF  PSF_FWHM     F_EFF  SKYBRIGHTNESS  B_EFF  CLOUD_APASS  CLOUD_NOMAD  N_APASS  N_NOMAD  C_EFF   SKYTILT  ASTROM_SIGMA  ASTROM_OFFSET  ASTROM_NDETS  ASTROM_CHI2  NOBJECTS  FLAG  CALNAC  CLOUD_DES  N_DES  CCDNUM                                   FILENAME      RAC1      DECC1      RAC2      DECC2       RAC3      DECC3       RAC4      DECC4   RA_CENT   DEC_CENT     RACMAX    RACMIN CROSSRA0
0  161052  9.352875 -44.129721    120.0     1.42    g  20121213  56275.145544  None    ES1    None        20940893  DECADE_FINALCUT  DECADE_FINALCUT  0.608711     1.139  0.708628           1.22  0.859       -0.002        0.099      418     1291    1.0  0.005376       0.04882       0.007071          2723    35.714001    129158     0       0     -0.013   9046       1  D00161052_g_c01_r4157p01_red-fullcat.fits  8.709225 -43.151698  8.707232 -43.300977

## 6. DES Year 6 Deep Field Exposure SE_Object Catalog Downloads fron NCSA

### 6.a. Create the Download Scripts

In [77]:
# Create main RawCatFITS directory, if it does not already exist...
rawcatfits_dir = os.path.join(output_dir,'RawCatFITS')
print rawcatfits_dir
if not os.path.exists(rawcatfits_dir):
    os.makedirs(rawcatfits_dir)

../RawData/RawCatFITS


**We will first work with the DES db, and then we will repeat for the DECADE db.**

**DES db:**

In [78]:
%%time

if do_create_des_curl_script:
    
    # Create main RawCatFITS DES subdirectory, if it does not already exist#...
    des_subdir = """DES_%s""" % (tag_des)
    rawcatfits_des_dir = os.path.join(rawcatfits_dir, des_subdir)
    print rawcatfits_des_dir
    if not os.path.exists(rawcatfits_des_dir):
        os.makedirs(rawcatfits_des_dir)
    
    # Create bash script to download the DES catalog files...
    bashScriptName = os.path.join(rawcatfits_des_dir, 'rawCatFits_des.bash')
    print bashScriptName
    fout = open(bashScriptName, 'w')
    for expnum in uniqExposureList:
        df3_des_deep_expnum = df3_des_deep[(df3_des_deep.EXPNUM == expnum)]
        for index, row in df3_des_deep_expnum.iterrows():
            clink = row['CLINK']
            filename = os.path.basename(clink)
            outputLine = """curl -n -o %s %s""" % (filename, clink)
            #print outputLine
            fout.write(outputLine+'\n')
    fout.close()

../RawData/RawCatFITS/DES_Y6A2_FINALCUT
../RawData/RawCatFITS/DES_Y6A2_FINALCUT/rawCatFits_des.bash
CPU times: user 2min 4s, sys: 4.04 s, total: 2min 8s
Wall time: 2min 16s


**DECADE db:**

In [79]:
%%time

if do_create_decade_curl_script:
    
    # Create main RawCatFITS DECADE subdirectory, if it does not already exist#...
    decade_subdir = """%s""" % (tag_decade)
    rawcatfits_decade_dir = os.path.join(rawcatfits_dir, decade_subdir)
    print rawcatfits_decade_dir
    if not os.path.exists(rawcatfits_decade_dir):
        os.makedirs(rawcatfits_decade_dir)
    
    # Create bash script to download the DECADE catalog files...
    bashScriptName = os.path.join(rawcatfits_decade_dir, 'rawCatFits_decade.bash')
    print bashScriptName
    fout = open(bashScriptName, 'w')
    for expnum in uniqExposureList:
        df3_decade_deep_expnum = df3_decade_deep[(df3_decade_deep.EXPNUM == expnum)]
        for index, row in df3_decade_deep_expnum.iterrows():
            clink = row['CLINK']
            filename = os.path.basename(clink)
            outputLine = """curl -n -o %s %s""" % (filename, clink)
            #print outputLine
            fout.write(outputLine+'\n')
    fout.close()

../RawData/RawCatFITS/DECADE_FINALCUT
../RawData/RawCatFITS/DECADE_FINALCUT/rawCatFits_decade.bash
CPU times: user 32.4 s, sys: 1.19 s, total: 33.6 s
Wall time: 34.9 s


### 6.b. Run the Download Scripts

**We will first work with the DES db, and then we will repeat for the DECADE db.**

** DES db:**

The Y6 Deep Field DES catalogs take about a day to download.

In [80]:
# To run rawCatFits_des.bash, you will need a .netrc file in your home directory containing
#  the appropriate connection info for the DES Y6A2 file database...

if do_run_des_curl_script:
    
    print """There's a way to do this within this Jupyter notebook, but, in the the end, it is easier to do the following."""
    print """In a terminal, cd into RawData/RawCatFITS/DES_%s and run the following command:""" % (tag_des)
    print """rm -f rawCatFits_des.log; source rawCatFits_des.bash > rawCatFits_des.log 2>&1 &"""

CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 22.2 µs


** DECADE db:**

The Y6 Deep Field DECADE catalogs take about 5 hours to download.

In [83]:
# To run rawCatFits_decade.bash, you will need a .netrc file in your home directory containing
#  the appropriate connection info for the DECADE file database...

if do_run_decade_curl_script:
    
    print """There's a way to do this within this Jupyter notebook, but, in the the end, it is easier to do the following."""
    print """In a terminal, cd into RawData/RawCatFITS/%s and run the following command:""" % (tag_decade)
    print """rm -f rawCatFits_decade.log; source rawCatFits_decade.bash > rawCatFits_decade.log 2>&1 &"""

CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 22.9 µs


In [84]:
if do_run_des_curl_script or do_run_decade_curl_script:
    
    print 'We will stop automatic execution beyone this point.'
    print 'Make sure all the catalog files from the previous two'
    print 'code cells have been copied over.  Once they have, '
    print 'you can continued with Step 7 and beyone (below).'
    
    # Stop automated execution beyond this point...
    raise StopExecution

## 7. Combine Exp/Image/QA data with SE_Object Catalogs

**We will first work with the DES data, and then we will repeat for the DECADE data.**

**DES:**

In [87]:
df345_des_deep.head(10)

,EXPNUM,EXPRA,EXPDEC,EXPTIME,AIRMASS,BAND,NITE,MJD_OBS,FIELD,OBJECT,PROGRAM,PFW_ATTEMPT_ID,TAG,SOURCE,T_EFF,PSF_FWHM,F_EFF,SKYBRIGHTNESS,B_EFF,CLOUD_APASS,CLOUD_NOMAD,N_APASS,N_NOMAD,C_EFF,SKYTILT,ASTROM_SIGMA,ASTROM_OFFSET,ASTROM_NDETS,ASTROM_CHI2,NOBJECTS,FLAG,CALNAC,CLOUD_DES,N_DES,CCDNUM,FILENAME,RAC1,DECC1,RAC2,DECC2,RAC3,DECC3,RAC4,DECC4,RA_CENT,DEC_CENT,RACMAX,RACMIN,CROSSRA0
0,154852,52.648333,-28.1,330.0,1.35,z,20121123,56255.03304,SN-C3,DES supernova hex SN-C3 tiling 76,supernova,2320793,Y6A1_FINALCUT,Y6A1_FINALCUT,0.302172,1.162,0.521885,28.540001,0.579,NaN,0.13,NaN,676.0,1.0,0.011746,0.035994,0.002818,3711.0,52.912998,117498.0,0,0,0.045,10201.0,1,D00154852_z_c01_r3727p02_red-fullcat.fits,52.131127,-27.120527,52.130154,-27.269728,52.466676,-27.270366,52.467110,-27.121305,52.298702,-27.195535,52.467110,52.130154,N
1,154852,52.648333,-28.1,330.0,1.35,z,20121123,56255.03304,SN-C3,DES supernova hex SN-C3 tiling 76,supernova,2320793,Y6A1_FINALCUT,Y6A1_FINALCUT,0.302172,1.162,0.521885,28.540001,0.579,NaN,0.13,NaN,676.0,1.0,0.011746,0.035994,0.002818,3711.0,52.912998,117498.0,0,0,0.045,10201.0,2,D00154852_z_c02_r3727p02_red-fullcat.fits,52.479785,-27.121378,52.479072,-27.270434,52.815817,-27.271100,52.815881,-27.122048,52.647600,-27.196260,52.815881,52.479072,N
2,154852,52.648333,-28.1,330.0,1.35,z,20121123,56255.03304,SN-C3,DES supernova hex SN-C3 tiling 76,supernova,2320793,Y6A1_FINALCUT,Y6A1_FINALCUT,0.302172,1.162,0.521885,28.540001,0.579,NaN,0.13,NaN,676.0,1.0,0.011746,0.035994,0.002818,3711.0,52.912998,117498.0,0,0,0.045,10201.0,3,D00154852_z_c03_r3727p02_red-fullcat.fits,52.828725,-27.121356,52.828955,-27.270409,53.165481,-27.270088,53.164714,-27.120895,52.996952,-27.195740,53.165481,52.828725,N
3,154852,52.648333,-28.1,330.0,1.35,z,20121123,56255.03304,SN-C3,DES supernova hex SN-C3 tiling 76,supernova,2320793,Y6A1_FINALCUT,Y6A1_FINALCUT,0.302172,1.162,0.521885,28.540001,0.579,NaN,0.13,NaN,676.0,1.0,0.011746,0.035994,0.002818,3711.0,52.912998,117498.0,0,0,0.045,10201.0,4,D00154852_z_c04_r3727p02_red-fullcat.fits,51.955577,-27.283827,51.954251,-27.433078,52.291139,-27.434045,52.291958,-27.284860,52.123132,-27.358997,52.291958,51.954251,N
4,154852,52.648333,-28.1,330.0,1.35,z,20121123,56255.03304,SN-C3,DES supernova hex SN-C3 tiling 76,supernova,2320793,Y6A1_FINALCUT,Y6A1_FINALCUT,0.302172,1.162,0.521885,28.540001,0.579,NaN,0.13,NaN,676.0,1.0,0.011746,0.035994,0.002818,3711.0,52.912998,117498.0,0,0,0.045,10201.0,5,D00154852_z_c05_r3727p02_red-fullcat.fits,52.304707,-27.284821,52.303990,-27.433994,52.641400,-27.434251,52.641447,-27.285078,52.472810,-27.359532,52.641447,52.303990,N
5,154852,52.648333,-28.1,330.0,1.35,z,20121123,56255.03304,SN-C3,DES supernova hex SN-C3 tiling 76,supernova,2320793,Y6A1_FINALCUT,Y6A1_FINALCUT,0.302172,1.162,0.521885,28.540001,0.579,NaN,0.13,NaN,676.0,1.0,0.011746,0.035994,0.002818,3711.0,52.912998,117498.0,0,0,0.045,10201.0,6,D00154852_z_c06_r3727p02_red-fullcat.fits,52.654231,-27.285113,52.654087,-27.434286,52.991499,-27.434309,52.990977,-27.285127,52.822691,-27.359704,52.991499,52.654087,N
6,154852,52.648333,-28.1,330.0,1.35,z,20121123,56255.03304,SN-C3,DES supernova hex SN-C3 tiling 76,supernova,2320793,Y6A1_FINALCUT,Y6A1_FINALCUT,0.302172,1.162,0.521885,28.540001,0.579,NaN,0.13,NaN,676.0,1.0,0.011746,0.035994,0.002818,3711.0,52.912998,117498.0,0,0,0.045,10201.0,7,D00154852_z_c07_r3727p02_red-fullcat.fits,53.003585,-27.285108,53.004206,-27.434287,53.341103,-27.433621,53.339969,-27.284375,53.172232,-27.359385,53.341103,53.003585,N
7,154852,52.648333,-28.1,330.0,1.35,z,20121123,56255.03304,SN-C3,DES supernova hex SN-C3 tiling 76,supernova,2320793,Y6A1_FINALCUT,Y6A1_FINALCUT,0.302172,1.162,0.521885,28.540001,0.579,NaN,0.13,NaN,676.0,1.0,0.011746,0.035994,0.002818,3711.0,52.912998,117498.0,0,0,0.045,10201.0,8,D00154852_z_c08_r3727p02_red-fullcat.fits,51.779568,-27.447229,51.778092,-27.596579,52.115186,-27.597805,52.116216,-27.448440,51.947148,-27.522564,52.116216,51.778092,N
8,154852,52.64

In [ ]:
%%time

if do_concatenate_des_se_objects:
    
    df_input = df345_des_deep
    subdir = """DES_%s""" % (tag_des)

    
    #df_input['FILENAME'] = df_input['FILEPATH'].apply(lambda x: os.path.basename(x))

    for band in bandList:
        
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""
    
        #i = 0
        df_cat_list = []
        mask_band = (df_input.BAND == band)
    
        for index, row in df_input[mask_band].iterrows():
            
            #if i > 100: break
            
            fileName = row['FILENAME']
            expnum = row['EXPNUM']
            ccdnum = row['CCDNUM']
            
            fullFileName = os.path.join(output_dir, 'RawCatFITS', subdir, fileName)

            if os.path.isfile(fullFileName):

                print fullFileName, expnum, ccdnum
                
                try:
            
                    df_cat = catfits_to_catdf(fullFileName, expnum, ccdnum)
                
                except:
                    
                    print fullFileName+':  Failed catfits_to_catdf'

                try:
                    
                    # Create mask analogous to the limits applied in the Y3A2_MISC SE_OBJECT query....
                    mask1 = (df_cat.X_IMAGE >= 100) & (df_cat.X_IMAGE <= 1900)
                    mask2 = (df_cat.Y_IMAGE >= 100) & (df_cat.Y_IMAGE <= 3950)
                    mask3 = (df_cat.FLAGS < 2) & (df_cat.IMAFLAGS_ISO < 2)
                    mask4 = (df_cat.SPREAD_MODEL >= -0.01) & (df_cat.SPREAD_MODEL <= 0.01)
                    mask5 = (df_cat.CLASS_STAR > 0.80)
                    magerr_psf = 1.086*df_cat.FLUXERR_PSF/df_cat.FLUX_PSF
                    mask6 =  (magerr_psf >= 0.0001) & (magerr_psf <= 0.1000)
                    mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6
                
                except:
                    
                    print fullFileName+':  Failed masking'

                try:
                    
                    if df_cat.EXPNUM.size > 0:
                        df_cat_tmp = df_cat[mask].copy()
                        df_cat_list.append(df_cat_tmp)
                        del df_cat
                    
                except:
                    
                    print fullFileName+':  Failed appending'
                
            else:
            
                print fullFileName+':  file does not exist/has problem'
                
            #i = i + 1 
        
        
        if verbose>0:
            print 'Concatenating individual files...'
        #print df_cat_list
        df_cat = pd.concat(df_cat_list, ignore_index=True)

        if verbose>0:
            print 'Merging exp/imginfo with se_obj info...'
        df_merged = pd.merge(df_input, df_cat, on=['FILENAME'], how='inner', suffixes=('','_y'))
                
        if verbose>0:
            print 'Removing any redundant columns...'
        to_drop = [colname for colname in df_merged if colname.endswith('_y')]
        df_merged.drop(to_drop, axis=1, inplace=True)
            
        if verbose>0:
            print 'Sorting merged dataframe by RA_WRAP...'
        df_merged = df_merged.sort_values(by=['RA_WRAP'])

        if verbose>0:
            print 'Resetting index...'
        df_merged.reset_index(drop=True)

        outputFile = """y6a1_rawdata.sorted.trimmed.%s.%s.csv""" % (subdir, band)
        outputFile = os.path.join(output_dir, outputFile)
        if verbose>0:
            print """Outputting merged CSV file %s...""" % (outputFile)
        df_merged.to_csv(outputFile, index=False)
        
        #inputFile = outputFile
        outputFile = """y6a1_rawdata.sorted.trimmed.%s.%s.fits""" % (subdir, band)
        outputFile = os.path.join(output_dir, outputFile)
        if verbose>0:
            print """Outputting merged FITS file %s...""" % (outputFile)
        #t = Table.read(inputFile, format='ascii.csv')
        t = Table.from_pandas(df_merged)
        t.write(outputFile, overwrite=True)
        del t        
        
        del df_merged
        
        gc.collect()


**DECADE:**

In [91]:
df345_decade_deep.head(10)

,EXPNUM,EXPRA,EXPDEC,EXPTIME,AIRMASS,BAND,NITE,MJD_OBS,FIELD,OBJECT,PROGRAM,PFW_ATTEMPT_ID,TAG,SOURCE,T_EFF,PSF_FWHM,F_EFF,SKYBRIGHTNESS,B_EFF,CLOUD_APASS,CLOUD_NOMAD,N_APASS,N_NOMAD,C_EFF,SKYTILT,ASTROM_SIGMA,ASTROM_OFFSET,ASTROM_NDETS,ASTROM_CHI2,NOBJECTS,FLAG,CALNAC,CLOUD_DES,N_DES,CCDNUM,FILENAME,RAC1,DECC1,RAC2,DECC2,RAC3,DECC3,RAC4,DECC4,RA_CENT,DEC_CENT,RACMAX,RACMIN,CROSSRA0
0,161052,9.352875,-44.129721,120.0,1.42,g,20121213,56275.145544,None,ES1,None,20940893,DECADE_FINALCUT,DECADE_FINALCUT,0.608711,1.139,0.708628,1.22,0.859,-0.002,0.099,418,1291,1.0,0.005376,0.04882,0.007071,2723,35.714001,129158,0,0,-0.013,9046,1,D00161052_g_c01_r4157p01_red-fullcat.fits,8.709225,-43.151698,8.707232,-43.300977,9.118308,-43.302602,9.119224,-43.153482,8.913427,-43.227339,9.119224,8.707232,N
1,161052,9.352875,-44.129721,120.0,1.42,g,20121213,56275.145544,None,ES1,None,20940893,DECADE_FINALCUT,DECADE_FINALCUT,0.608711,1.139,0.708628,1.22,0.859,-0.002,0.099,418,1291,1.0,0.005376,0.04882,0.007071,2723,35.714001,129158,0,0,-0.013,9046,2,D00161052_g_c02_r4157p01_red-fullcat.fits,9.134688,-43.153579,9.133448,-43.302691,9.544778,-43.303619,9.544796,-43.154509,9.339379,-43.228715,9.544796,9.133448,N
2,161052,9.352875,-44.129721,120.0,1.42,g,20121213,56275.145544,None,ES1,None,20940893,DECADE_FINALCUT,DECADE_FINALCUT,0.608711,1.139,0.708628,1.22,0.859,-0.002,0.099,418,1291,1.0,0.005376,0.04882,0.007071,2723,35.714001,129158,0,0,-0.013,9046,3,D00161052_g_c03_r4157p01_red-fullcat.fits,9.560469,-43.153812,9.560824,-43.302922,9.971903,-43.302133,9.970473,-43.152866,9.765891,-43.228082,9.971903,9.560469,N
3,161052,9.352875,-44.129721,120.0,1.42,g,20121213,56275.145544,None,ES1,None,20940893,DECADE_FINALCUT,DECADE_FINALCUT,0.608711,1.139,0.708628,1.22,0.859,-0.002,0.099,418,1291,1.0,0.005376,0.04882,0.007071,2723,35.714001,129158,0,0,-0.013,9046,4,D00161052_g_c04_r4157p01_red-fullcat.fits,8.493856,-43.314283,8.491183,-43.463578,8.903202,-43.465881,8.904829,-43.316672,8.698157,-43.390243,8.904829,8.491183,N
4,161052,9.352875,-44.129721,120.0,1.42,g,20121213,56275.145544,None,ES1,None,20940893,DECADE_FINALCUT,DECADE_FINALCUT,0.608711,1.139,0.708628,1.22,0.859,-0.002,0.099,418,1291,1.0,0.005376,0.04882,0.007071,2723,35.714001,129158,0,0,-0.013,9046,5,D00161052_g_c05_r4157p01_red-fullcat.fits,8.920403,-43.316674,8.918917,-43.465867,9.331549,-43.466742,9.331771,-43.317551,9.125569,-43.391794,9.331771,8.918917,N
5,161052,9.352875,-44.129721,120.0,1.42,g,20121213,56275.145544,None,ES1,None,20940893,DECADE_FINALCUT,DECADE_FINALCUT,0.608711,1.139,0.708628,1.22,0.859,-0.002,0.099,418,1291,1.0,0.005376,0.04882,0.007071,2723,35.714001,129158,0,0,-0.013,9046,6,D00161052_g_c06_r4157p01_red-fullcat.fits,9.347387,-43.317595,9.347063,-43.466786,9.759700,-43.466713,9.758767,-43.317509,9.553217,-43.392236,9.759700,9.347063,N
6,161052,9.352875,-44.129721,120.0,1.42,g,20121213,56275.145544,None,ES1,None,20940893,DECADE_FINALCUT,DECADE_FINALCUT,0.608711,1.139,0.708628,1.22,0.859,-0.002,0.099,418,1291,1.0,0.005376,0.04882,0.007071,2723,35.714001,129158,0,0,-0.013,9046,7,D00161052_g_c07_r4157p01_red-fullcat.fits,9.774170,-43.317475,9.775241,-43.466676,10.187274,-43.465194,10.185149,-43.315907,9.980469,-43.391443,10.187274,9.774170,N
7,161052,9.352875,-44.129721,120.0,1.42,g,20121213,56275.145544,None,ES1,None,20940893,DECADE_FINALCUT,DECADE_FINALCUT,0.608711,1.139,0.708628,1.22,0.859,-0.002,0.099,418,1291,1.0,0.005376,0.04882,0.007071,2723,35.714001,129158,0,0,-0.013,9046,8,D00161052_g_c08_r4157p01_red-fullcat.fits,8.277382,-43.476758,8.274273,-43.626127,8.687057,-43.629041,8.689189,-43.479674,8.481846,-43.553042,8.689189,8.274273,N
8,161052,9.352875,-44.129721,120.0,1.42,g,20121213,56275.145544,None,ES1,None,20940893,DECADE_FINALCUT,DECADE_FINALCUT,0.608711,1.139,0.708628,1.22,0.859,-0.002,0.099,418,1291,1.0,0.005376,0.04882,0.007071,2723,35.714001,129158,0,0,-0.013,9046,9,D00161052_g_c09_r4157p01_red-fullcat.fits,8.704798,-43.479601,8.702494,-43.628962,9.116193,-43.630781

In [ ]:
%%time

if do_concatenate_decade_se_objects:
    
    df_input = df345_decade_deep
    subdir = """%s""" % (tag_decade)

    
    #df_input['FILENAME'] = df_input['FILEPATH'].apply(lambda x: os.path.basename(x))

    for band in bandList:
        
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""
    
        #i = 0
        df_cat_list = []
        mask_band = (df_input.BAND == band)
    
        for index, row in df_input[mask_band].iterrows():
            
            #if i > 100: break
            
            fileName = row['FILENAME']
            expnum = row['EXPNUM']
            ccdnum = row['CCDNUM']
            
            fullFileName = os.path.join(output_dir, 'RawCatFITS', subdir, fileName)

            if os.path.isfile(fullFileName):

                print fullFileName, expnum, ccdnum
                
                try:
            
                    df_cat = catfits_to_catdf(fullFileName, expnum, ccdnum)
                
                except:
                    
                    print fullFileName+':  Failed catfits_to_catdf'

                try:
                    
                    # Create mask analogous to the limits applied in the Y3A2_MISC SE_OBJECT query....
                    mask1 = (df_cat.X_IMAGE >= 100) & (df_cat.X_IMAGE <= 1900)
                    mask2 = (df_cat.Y_IMAGE >= 100) & (df_cat.Y_IMAGE <= 3950)
                    mask3 = (df_cat.FLAGS < 2) & (df_cat.IMAFLAGS_ISO < 2)
                    mask4 = (df_cat.SPREAD_MODEL >= -0.01) & (df_cat.SPREAD_MODEL <= 0.01)
                    mask5 = (df_cat.CLASS_STAR > 0.80)
                    magerr_psf = 1.086*df_cat.FLUXERR_PSF/df_cat.FLUX_PSF
                    mask6 =  (magerr_psf >= 0.0001) & (magerr_psf <= 0.1000)
                    mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6
                
                except:
                    
                    print fullFileName+':  Failed masking'

                try:
                    
                    if df_cat.EXPNUM.size > 0:
                        df_cat_tmp = df_cat[mask].copy()
                        df_cat_list.append(df_cat_tmp)
                        del df_cat
                    
                except:
                    
                    print fullFileName+':  Failed appending'
                
            else:
            
                print fullFileName+':  file does not exist/has problem'
                
            #i = i + 1 
        
        
        if verbose>0:
            print 'Concatenating individual files...'
        #print df_cat_list
        df_cat = pd.concat(df_cat_list, ignore_index=True)

        if verbose>0:
            print 'Merging exp/imginfo with se_obj info...'
        df_merged = pd.merge(df_input, df_cat, on=['FILENAME'], how='inner', suffixes=('','_y'))
                
        if verbose>0:
            print 'Removing any redundant columns...'
        to_drop = [colname for colname in df_merged if colname.endswith('_y')]
        df_merged.drop(to_drop, axis=1, inplace=True)
            
        if verbose>0:
            print 'Sorting merged dataframe by RA_WRAP...'
        df_merged = df_merged.sort_values(by=['RA_WRAP'])

        if verbose>0:
            print 'Resetting index...'
        df_merged.reset_index(drop=True)

        outputFile = """y6a1_rawdata.sorted.trimmed.%s.%s.csv""" % (subdir, band)
        outputFile = os.path.join(output_dir, outputFile)
        if verbose>0:
            print """Outputting merged CSV file %s...""" % (outputFile)
        df_merged.to_csv(outputFile, index=False)
        
        # Having some trouble doing a direct Table to FITS conversion; 
        #  so will just read in the CSV file and write out a FITS table
        inputFile = outputFile
        outputFile = """y6a1_rawdata.sorted.trimmed.%s.%s.fits""" % (subdir, band)
        outputFile = os.path.join(output_dir, outputFile)
        if verbose>0:
            print """Outputting merged FITS file %s...""" % (outputFile)
        t = Table.read(inputFile, format='ascii.csv')
        #t = Table.from_pandas(df_merged)
        t.write(outputFile, format='fits', overwrite=True)
        del t        
        
        del df_merged
        
        gc.collect()


# # # # # # # # # # # # # # #
Working on band u
# # # # # # # # # # # # # # #
../RawData/RawCatFITS/DECADE_FINALCUT/D00945150_u_c01_r5153p02_red-fullcat.fits 945150 1
../RawData/RawCatFITS/DECADE_FINALCUT/D00945150_u_c03_r5153p02_red-fullcat.fits 945150 3
../RawData/RawCatFITS/DECADE_FINALCUT/D00945150_u_c04_r5153p02_red-fullcat.fits 945150 4
../RawData/RawCatFITS/DECADE_FINALCUT/D00945150_u_c05_r5153p02_red-fullcat.fits 945150 5
../RawData/RawCatFITS/DECADE_FINALCUT/D00945150_u_c06_r5153p02_red-fullcat.fits 945150 6
../RawData/RawCatFITS/DECADE_FINALCUT/D00945150_u_c07_r5153p02_red-fullcat.fits 945150 7
../RawData/RawCatFITS/DECADE_FINALCUT/D00945150_u_c08_r5153p02_red-fullcat.fits 945150 8
../RawData/RawCatFITS/DECADE_FINALCUT/D00945150_u_c09_r5153p02_red-fullcat.fits 945150 9
../RawData/RawCatFITS/DECADE_FINALCUT/D00945150_u_c10_r5153p02_red-fullcat.fits 945150 10
../RawData/RawCatFITS/DECADE_FINALCUT/D00945150_u_c11_r5153p02_red-fullcat.fits 945150 11
../RawData/RawCatFITS/DECADE_F

../RawData/RawCatFITS/DECADE_FINALCUT/D00945151_u_c34_r5147p03_red-fullcat.fits 945151 34
../RawData/RawCatFITS/DECADE_FINALCUT/D00945151_u_c35_r5147p03_red-fullcat.fits 945151 35
../RawData/RawCatFITS/DECADE_FINALCUT/D00945151_u_c36_r5147p03_red-fullcat.fits 945151 36
../RawData/RawCatFITS/DECADE_FINALCUT/D00945151_u_c37_r5147p03_red-fullcat.fits 945151 37
../RawData/RawCatFITS/DECADE_FINALCUT/D00945151_u_c38_r5147p03_red-fullcat.fits 945151 38
../RawData/RawCatFITS/DECADE_FINALCUT/D00945151_u_c39_r5147p03_red-fullcat.fits 945151 39
../RawData/RawCatFITS/DECADE_FINALCUT/D00945151_u_c40_r5147p03_red-fullcat.fits 945151 40
../RawData/RawCatFITS/DECADE_FINALCUT/D00945151_u_c41_r5147p03_red-fullcat.fits 945151 41
../RawData/RawCatFITS/DECADE_FINALCUT/D00945151_u_c42_r5147p03_red-fullcat.fits 945151 42
../RawData/RawCatFITS/DECADE_FINALCUT/D00945151_u_c43_r5147p03_red-fullcat.fits 945151 43
../RawData/RawCatFITS/DECADE_FINALCUT/D00945151_u_c44_r5147p03_red-fullcat.fits 945151 44
../RawData

../RawData/RawCatFITS/DECADE_FINALCUT/D00945153_u_c07_r5147p03_red-fullcat.fits 945153 7
../RawData/RawCatFITS/DECADE_FINALCUT/D00945153_u_c08_r5147p03_red-fullcat.fits 945153 8
../RawData/RawCatFITS/DECADE_FINALCUT/D00945153_u_c09_r5147p03_red-fullcat.fits 945153 9
../RawData/RawCatFITS/DECADE_FINALCUT/D00945153_u_c10_r5147p03_red-fullcat.fits 945153 10
../RawData/RawCatFITS/DECADE_FINALCUT/D00945153_u_c11_r5147p03_red-fullcat.fits 945153 11
../RawData/RawCatFITS/DECADE_FINALCUT/D00945153_u_c12_r5147p03_red-fullcat.fits 945153 12
../RawData/RawCatFITS/DECADE_FINALCUT/D00945153_u_c13_r5147p03_red-fullcat.fits 945153 13
../RawData/RawCatFITS/DECADE_FINALCUT/D00945153_u_c14_r5147p03_red-fullcat.fits 945153 14
../RawData/RawCatFITS/DECADE_FINALCUT/D00945153_u_c15_r5147p03_red-fullcat.fits 945153 15
../RawData/RawCatFITS/DECADE_FINALCUT/D00945153_u_c16_r5147p03_red-fullcat.fits 945153 16
../RawData/RawCatFITS/DECADE_FINALCUT/D00945153_u_c17_r5147p03_red-fullcat.fits 945153 17
../RawData/Ra

../RawData/RawCatFITS/DECADE_FINALCUT/D00945154_u_c40_r5147p03_red-fullcat.fits 945154 40
../RawData/RawCatFITS/DECADE_FINALCUT/D00945154_u_c41_r5147p03_red-fullcat.fits 945154 41
../RawData/RawCatFITS/DECADE_FINALCUT/D00945154_u_c42_r5147p03_red-fullcat.fits 945154 42
../RawData/RawCatFITS/DECADE_FINALCUT/D00945154_u_c43_r5147p03_red-fullcat.fits 945154 43
../RawData/RawCatFITS/DECADE_FINALCUT/D00945154_u_c44_r5147p03_red-fullcat.fits 945154 44
../RawData/RawCatFITS/DECADE_FINALCUT/D00945154_u_c45_r5147p03_red-fullcat.fits 945154 45
../RawData/RawCatFITS/DECADE_FINALCUT/D00945154_u_c46_r5147p03_red-fullcat.fits 945154 46
../RawData/RawCatFITS/DECADE_FINALCUT/D00945154_u_c47_r5147p03_red-fullcat.fits 945154 47
../RawData/RawCatFITS/DECADE_FINALCUT/D00945154_u_c48_r5147p03_red-fullcat.fits 945154 48
../RawData/RawCatFITS/DECADE_FINALCUT/D00945154_u_c49_r5147p03_red-fullcat.fits 945154 49
../RawData/RawCatFITS/DECADE_FINALCUT/D00945154_u_c50_r5147p03_red-fullcat.fits 945154 50
../RawData

../RawData/RawCatFITS/DECADE_FINALCUT/D00945156_u_c14_r5147p03_red-fullcat.fits 945156 14
../RawData/RawCatFITS/DECADE_FINALCUT/D00945156_u_c15_r5147p03_red-fullcat.fits 945156 15
../RawData/RawCatFITS/DECADE_FINALCUT/D00945156_u_c16_r5147p03_red-fullcat.fits 945156 16
../RawData/RawCatFITS/DECADE_FINALCUT/D00945156_u_c17_r5147p03_red-fullcat.fits 945156 17
../RawData/RawCatFITS/DECADE_FINALCUT/D00945156_u_c18_r5147p03_red-fullcat.fits 945156 18
../RawData/RawCatFITS/DECADE_FINALCUT/D00945156_u_c19_r5147p03_red-fullcat.fits 945156 19
../RawData/RawCatFITS/DECADE_FINALCUT/D00945156_u_c20_r5147p03_red-fullcat.fits 945156 20
../RawData/RawCatFITS/DECADE_FINALCUT/D00945156_u_c21_r5147p03_red-fullcat.fits 945156 21
../RawData/RawCatFITS/DECADE_FINALCUT/D00945156_u_c22_r5147p03_red-fullcat.fits 945156 22
../RawData/RawCatFITS/DECADE_FINALCUT/D00945156_u_c23_r5147p03_red-fullcat.fits 945156 23
../RawData/RawCatFITS/DECADE_FINALCUT/D00945156_u_c24_r5147p03_red-fullcat.fits 945156 24
../RawData

...

../RawData/RawCatFITS/DECADE_FINALCUT/D00457185_r_c35_r4644p01_red-fullcat.fits 457185 35
../RawData/RawCatFITS/DECADE_FINALCUT/D00457185_r_c36_r4644p01_red-fullcat.fits 457185 36
../RawData/RawCatFITS/DECADE_FINALCUT/D00457185_r_c37_r4644p01_red-fullcat.fits 457185 37
../RawData/RawCatFITS/DECADE_FINALCUT/D00457185_r_c38_r4644p01_red-fullcat.fits 457185 38
../RawData/RawCatFITS/DECADE_FINALCUT/D00457185_r_c39_r4644p01_red-fullcat.fits 457185 39
../RawData/RawCatFITS/DECADE_FINALCUT/D00457185_r_c40_r4644p01_red-fullcat.fits 457185 40
../RawData/RawCatFITS/DECADE_FINALCUT/D00457185_r_c41_r4644p01_red-fullcat.fits 457185 41
../RawData/RawCatFITS/DECADE_FINALCUT/D00457185_r_c42_r4644p01_red-fullcat.fits 457185 42
../RawData/RawCatFITS/DECADE_FINALCUT/D00457185_r_c43_r4644p01_red-fullcat.fits 457185 43
../RawData/RawCatFITS/DECADE_FINALCUT/D00457185_r_c44_r4644p01_red-fullcat.fits 457185 44
../RawData/RawCatFITS/DECADE_FINALCUT/D00457185_r_c45_r4644p01_red-fullcat.fits 457185 45
../RawData

../RawData/RawCatFITS/DECADE_FINALCUT/D00457187_r_c07_r4644p01_red-fullcat.fits 457187 7
../RawData/RawCatFITS/DECADE_FINALCUT/D00457187_r_c08_r4644p01_red-fullcat.fits 457187 8
../RawData/RawCatFITS/DECADE_FINALCUT/D00457187_r_c09_r4644p01_red-fullcat.fits 457187 9
../RawData/RawCatFITS/DECADE_FINALCUT/D00457187_r_c10_r4644p01_red-fullcat.fits 457187 10
../RawData/RawCatFITS/DECADE_FINALCUT/D00457187_r_c11_r4644p01_red-fullcat.fits 457187 11
../RawData/RawCatFITS/DECADE_FINALCUT/D00457187_r_c12_r4644p01_red-fullcat.fits 457187 12
../RawData/RawCatFITS/DECADE_FINALCUT/D00457187_r_c13_r4644p01_red-fullcat.fits 457187 13
../RawData/RawCatFITS/DECADE_FINALCUT/D00457187_r_c14_r4644p01_red-fullcat.fits 457187 14
../RawData/RawCatFITS/DECADE_FINALCUT/D00457187_r_c15_r4644p01_red-fullcat.fits 457187 15
../RawData/RawCatFITS/DECADE_FINALCUT/D00457187_r_c16_r4644p01_red-fullcat.fits 457187 16
../RawData/RawCatFITS/DECADE_FINALCUT/D00457187_r_c17_r4644p01_red-fullcat.fits 457187 17
../RawData/Ra

../RawData/RawCatFITS/DECADE_FINALCUT/D00457188_r_c39_r4644p01_red-fullcat.fits 457188 39
../RawData/RawCatFITS/DECADE_FINALCUT/D00457188_r_c40_r4644p01_red-fullcat.fits 457188 40
../RawData/RawCatFITS/DECADE_FINALCUT/D00457188_r_c41_r4644p01_red-fullcat.fits 457188 41
../RawData/RawCatFITS/DECADE_FINALCUT/D00457188_r_c42_r4644p01_red-fullcat.fits 457188 42
../RawData/RawCatFITS/DECADE_FINALCUT/D00457188_r_c43_r4644p01_red-fullcat.fits 457188 43
../RawData/RawCatFITS/DECADE_FINALCUT/D00457188_r_c44_r4644p01_red-fullcat.fits 457188 44
../RawData/RawCatFITS/DECADE_FINALCUT/D00457188_r_c45_r4644p01_red-fullcat.fits 457188 45
../RawData/RawCatFITS/DECADE_FINALCUT/D00457188_r_c46_r4644p01_red-fullcat.fits 457188 46
../RawData/RawCatFITS/DECADE_FINALCUT/D00457188_r_c47_r4644p01_red-fullcat.fits 457188 47
../RawData/RawCatFITS/DECADE_FINALCUT/D00457188_r_c48_r4644p01_red-fullcat.fits 457188 48
../RawData/RawCatFITS/DECADE_FINALCUT/D00457188_r_c49_r4644p01_red-fullcat.fits 457188 49
../RawData

../RawData/RawCatFITS/DECADE_FINALCUT/D00457190_r_c12_r4644p01_red-fullcat.fits 457190 12
../RawData/RawCatFITS/DECADE_FINALCUT/D00457190_r_c13_r4644p01_red-fullcat.fits 457190 13
../RawData/RawCatFITS/DECADE_FINALCUT/D00457190_r_c14_r4644p01_red-fullcat.fits 457190 14
../RawData/RawCatFITS/DECADE_FINALCUT/D00457190_r_c15_r4644p01_red-fullcat.fits 457190 15
../RawData/RawCatFITS/DECADE_FINALCUT/D00457190_r_c16_r4644p01_red-fullcat.fits 457190 16
../RawData/RawCatFITS/DECADE_FINALCUT/D00457190_r_c17_r4644p01_red-fullcat.fits 457190 17
../RawData/RawCatFITS/DECADE_FINALCUT/D00457190_r_c18_r4644p01_red-fullcat.fits 457190 18
../RawData/RawCatFITS/DECADE_FINALCUT/D00457190_r_c19_r4644p01_red-fullcat.fits 457190 19
../RawData/RawCatFITS/DECADE_FINALCUT/D00457190_r_c20_r4644p01_red-fullcat.fits 457190 20
../RawData/RawCatFITS/DECADE_FINALCUT/D00457190_r_c21_r4644p01_red-fullcat.fits 457190 21
../RawData/RawCatFITS/DECADE_FINALCUT/D00457190_r_c22_r4644p01_red-fullcat.fits 457190 22
../RawData

../RawData/RawCatFITS/DECADE_FINALCUT/D00457191_r_c44_r4644p01_red-fullcat.fits 457191 44
../RawData/RawCatFITS/DECADE_FINALCUT/D00457191_r_c45_r4644p01_red-fullcat.fits 457191 45
../RawData/RawCatFITS/DECADE_FINALCUT/D00457191_r_c46_r4644p01_red-fullcat.fits 457191 46
../RawData/RawCatFITS/DECADE_FINALCUT/D00457191_r_c47_r4644p01_red-fullcat.fits 457191 47
../RawData/RawCatFITS/DECADE_FINALCUT/D00457191_r_c48_r4644p01_red-fullcat.fits 457191 48
../RawData/RawCatFITS/DECADE_FINALCUT/D00457191_r_c49_r4644p01_red-fullcat.fits 457191 49
../RawData/RawCatFITS/DECADE_FINALCUT/D00457191_r_c50_r4644p01_red-fullcat.fits 457191 50
../RawData/RawCatFITS/DECADE_FINALCUT/D00457191_r_c51_r4644p01_red-fullcat.fits 457191 51
../RawData/RawCatFITS/DECADE_FINALCUT/D00457191_r_c52_r4644p01_red-fullcat.fits 457191 52
../RawData/RawCatFITS/DECADE_FINALCUT/D00457191_r_c53_r4644p01_red-fullcat.fits 457191 53
../RawData/RawCatFITS/DECADE_FINALCUT/D00457191_r_c54_r4644p01_red-fullcat.fits 457191 54
../RawData

## 9. Match with GALEX

See /data/des40.a/data/dtucker/SDSS/sdss_gaiaedr3_galex_match.bash for next steps.

**We will first work with the DES data, and then we will repeat for the DECADE data.**

**DES:**

In [ ]:
%%time 


if do_match_des_galex: 
    
    subdir = """DES_%s""" % (tag_des)
    
    tmpdir = os.environ['TMPDIR']
    band = 'u'
    inputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s.%s.fits""" % (subdir, band)
    outputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s_galex.%s.fits""" % (subdir, band)
    
    cmd = \
        """
        %s \\
        -Djava.io.tmpdir=%s -Xmx20000m cdsskymatch \\
        cdstable=II/335/galex_ais find=each radius=3.0 \\
        in=%s ifmt=fits ra=RA dec=DEC \\
        out=%s ofmt=fits
        """ % (stilts_exe, tmpdir, inputFile, outputFile)

    print cmd
    print os.system(cmd)

**DECADE:**

In [ ]:
%%time 


if do_match_decade_galex: 
    
    subdir = """%s""" % (tag_decade)
    
    tmpdir = os.environ['TMPDIR']
    band = 'u'
    inputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s.%s.fits""" % (subdir, band)
    outputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s_galex.%s.fits""" % (subdir, band)
    
    cmd = \
        """
        %s \\
        -Djava.io.tmpdir=%s -Xmx20000m cdsskymatch \\
        cdstable=II/335/galex_ais find=each radius=3.0 \\
        in=%s ifmt=fits ra=RA dec=DEC \\
        out=%s ofmt=fits
        """ % (stilts_exe, tmpdir, inputFile, outputFile)

    print cmd
    print os.system(cmd)

## 10. Match with GAIA EDR3

See /data/des40.a/data/dtucker/SDSS/sdss_gaiaedr3_galex_match.bash for next steps.

**We will first work with the DES data, and then we will repeat for the DECADE data.**

**DES:**

In [ ]:
%%time 

if do_match_des_gaiaedr3: 
    
    subdir = """DES_%s""" % (tag_des)

    for band in bandList:
        
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""

        tmpdir = os.environ['TMPDIR']

        inputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s.%s.fits""" % (subdir, band)
        outputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s_gaiaedr3.%s.fits""" % (subdir, band)
    
        cmd = \
            """
            %s \\
            -Djava.io.tmpdir=%s -Xmx20000m cdsskymatch \\
            cdstable=GAIA_EDR3_J2016 find=each radius=1.0 \\
            in=%s ifmt=fits ra=RA dec=DEC \\
            out=%s ofmt=fits
            """ % (stilts_exe, tmpdir, inputFile, outputFile)

            #cdstable=I/350/gaiaedr3 find=each radius=1.0 \\
            #cdstable=II/335/galex_ais find=each radius=3.0 \\

        print cmd
        print os.system(cmd)

In [ ]:
%%time 

# For u-band, we will also match the previously GALEX-matched catalog with GAIA EDR3...

if do_match_des_gaiaedr3: 

    subdir = """DES_%s""" % (tag_des)
    
    tmpdir = os.environ['TMPDIR']
    band = 'u'
    inputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s_galex.%s.fits""" % (subdir, band)
    outputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s_galex_gaiaedr3.%s.fits""" % (subdir, band)
    
    cmd = \
        """
        %s \\
        -Djava.io.tmpdir=%s -Xmx20000m cdsskymatch \\
        cdstable=GAIA_EDR3_J2016 find=each radius=1.0 \\
        in=%s ifmt=fits ra=RA dec=DEC \\
        out=%s ofmt=fits
        """ % (stilts_exe, tmpdir, inputFile, outputFile)

    print cmd
    print os.system(cmd)

**DECADE:**

In [ ]:
%%time 

if do_match_decade_gaiaedr3: 
    
    subdir = """%s""" % (tag_decade)

    for band in bandList:
        
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""

        tmpdir = os.environ['TMPDIR']

        inputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s.%s.fits""" % (subdir, band)
        outputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s_gaiaedr3.%s.fits""" % (subdir, band)
    
        cmd = \
            """
            %s \\
            -Djava.io.tmpdir=%s -Xmx20000m cdsskymatch \\
            cdstable=GAIA_EDR3_J2016 find=each radius=1.0 \\
            in=%s ifmt=fits ra=RA dec=DEC \\
            out=%s ofmt=fits
            """ % (stilts_exe, tmpdir, inputFile, outputFile)

            #cdstable=I/350/gaiaedr3 find=each radius=1.0 \\
            #cdstable=II/335/galex_ais find=each radius=3.0 \\

        print cmd
        print os.system(cmd)

In [ ]:
%%time 

# For u-band, we will also match the previously GALEX-matched catalog with GAIA EDR3...

if do_match_decade_gaiaedr3: 

    subdir = """%s""" % (tag_decade)
    
    tmpdir = os.environ['TMPDIR']
    band = 'u'
    inputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s_galex.%s.fits""" % (subdir, band)
    outputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s_galex_gaiaedr3.%s.fits""" % (subdir, band)
    
    cmd = \
        """
        %s \\
        -Djava.io.tmpdir=%s -Xmx20000m cdsskymatch \\
        cdstable=GAIA_EDR3_J2016 find=each radius=1.0 \\
        in=%s ifmt=fits ra=RA dec=DEC \\
        out=%s ofmt=fits
        """ % (stilts_exe, tmpdir, inputFile, outputFile)

    print cmd
    print os.system(cmd)

## 11. Match to FGCM Standard Stars (PSF)

**We will first work with the DES data, and then we will repeat for the DECADE data.**

**DES:**

In [ ]:
%%time 

if do_match_des_fgcm_psf: 
    
    subdir = """DES_%s""" % (tag_des)
    
    for band in bandList:
        
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""

        tmpdir = os.environ['TMPDIR']

        inputFileObs = """../RawData/y6a1_rawdata.sorted.trimmed.%s_gaiaedr3.%s.fits""" % (subdir, band)
        inputFileStd = '/data/des40.a/data/dtucker/Y6A1/StdStars/Y6A1_FGCM_V3_3_1_PSF_ALL_STARS.sorted.fits'
        outputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s_gaiaedr3_fgcm_psf.%s.fits""" % (subdir, band)
        
        cmd = """%s -Djava.io.tmpdir=%s -Xmx20000m tmatch2 matcher=sky find=best1 join=all1 params=1.0 in1=%s ifmt1=fits values1="RA_in DEC_in" in2=%s ifmt2=fits values2="RA DEC" out=%s ofmt=fits""" % (stilts_exe, tmpdir, inputFileObs, inputFileStd, outputFile)
        #ocmd='addcol -before 1 MATCHID $0' 
        #ocmd='addcol -before 1 MATCHID $0' fixcols=all 

            
        print cmd
        print os.system(cmd)

In [ ]:
%%time 

# For u-band, we will also match the previously GALEX- *and* GAIA EDR3-matched catalog with the FGCM standard stars...

if do_match_des_fgcm_psf: 
    
    subdir = """DES_%s""" % (tag_des)

    tmpdir = os.environ['TMPDIR']
    band = 'u'

    inputFileObs = """../RawData/y6a1_rawdata.sorted.trimmed.%s_galex_gaiaedr3.%s.fits""" % (subdir, band)
    inputFileStd = '/data/des40.a/data/dtucker/Y6A1/StdStars/Y6A1_FGCM_V3_3_1_PSF_ALL_STARS.sorted.fits'
    outputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s_galex_gaiaedr3_fgcm_psf.%s.fits""" % (subdir, band)
        
    cmd = """%s -Djava.io.tmpdir=%s -Xmx20000m tmatch2 matcher=sky find=best1 join=all1 params=1.0 in1=%s ifmt1=fits values1="RA_in DEC_in" in2=%s ifmt2=fits values2="RA DEC" out=%s ofmt=fits""" % (stilts_exe, tmpdir, inputFileObs, inputFileStd, outputFile)
    #ocmd='addcol -before 1 MATCHID $0' 
    #ocmd='addcol -before 1 MATCHID $0' fixcols=all 

            
    print cmd
    print os.system(cmd)

**DECADE:**

In [ ]:
%%time 

if do_match_decade_fgcm_psf: 
    
    subdir = """%s""" % (tag_decade)
    
    for band in bandList:
        
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""

        tmpdir = os.environ['TMPDIR']

        inputFileObs = """../RawData/y6a1_rawdata.sorted.trimmed.%s_gaiaedr3.%s.fits""" % (subdir, band)
        inputFileStd = '/data/des40.a/data/dtucker/Y6A1/StdStars/Y6A1_FGCM_V3_3_1_PSF_ALL_STARS.sorted.fits'
        outputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s_gaiaedr3_fgcm_psf.%s.fits""" % (subdir, band)
        
        cmd = """%s -Djava.io.tmpdir=%s -Xmx20000m tmatch2 matcher=sky find=best1 join=all1 params=1.0 in1=%s ifmt1=fits values1="RA_in DEC_in" in2=%s ifmt2=fits values2="RA DEC" out=%s ofmt=fits""" % (stilts_exe, tmpdir, inputFileObs, inputFileStd, outputFile)
        #ocmd='addcol -before 1 MATCHID $0' 
        #ocmd='addcol -before 1 MATCHID $0' fixcols=all 

            
        print cmd
        print os.system(cmd)

In [ ]:
%%time 

# For u-band, we will also match the previously GALEX- *and* GAIA EDR3-matched catalog with the FGCM standard stars...

if do_match_decade_fgcm_psf: 
    
    subdir = """%s""" % (tag_decade)

    tmpdir = os.environ['TMPDIR']
    band = 'u'

    inputFileObs = """../RawData/y6a1_rawdata.sorted.trimmed.%s_galex_gaiaedr3.%s.fits""" % (subdir, band)
    inputFileStd = '/data/des40.a/data/dtucker/Y6A1/StdStars/Y6A1_FGCM_V3_3_1_PSF_ALL_STARS.sorted.fits'
    outputFile = """../RawData/y6a1_rawdata.sorted.trimmed.%s_galex_gaiaedr3_fgcm_psf.%s.fits""" % (subdir, band)
        
    cmd = """%s -Djava.io.tmpdir=%s -Xmx20000m tmatch2 matcher=sky find=best1 join=all1 params=1.0 in1=%s ifmt1=fits values1="RA_in DEC_in" in2=%s ifmt2=fits values2="RA DEC" out=%s ofmt=fits""" % (stilts_exe, tmpdir, inputFileObs, inputFileStd, outputFile)
    #ocmd='addcol -before 1 MATCHID $0' 
    #ocmd='addcol -before 1 MATCHID $0' fixcols=all 

            
    print cmd
    print os.system(cmd)

## 12. Match to FGCM Standard Stars (APER_08)

**We will first work with the DES data, and then we will repeat for the DECADE data.**

**DES:**

In [ ]:
%%time 

if do_match_des_fgcm_aper8: 
    
    subdir = """DES_%s""" % (tag_des)

    for band in bandList:
        
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""

        tmpdir = os.environ['TMPDIR']

        inputFileObs = """../RawData/y6a1_rawdata.sorted.trimmed.y6_gaiaedr3.%s.fits""" % (subdir, band)
        inputFileStd = '/data/des40.a/data/dtucker/Y6A1/StdStars/Y6A1_FGCM_V2_1_2_2_ALL_STARS.sorted.fits'
        outputFile = """../RawData/y6a1_rawdata.sorted.trimmed.y6_gaiaedr3_fgcm_aper8.%s.fits""" % (subdir, band)
        
        cmd = """%s -Djava.io.tmpdir=%s -Xmx20000m tmatch2 matcher=sky find=best1 join=all1 params=1.0 in1=%s ifmt1=fits values1="RA_in DEC_in" in2=%s ifmt2=fits values2="RA DEC" out=%s ofmt=fits""" % (stilts_exe, tmpdir, inputFileObs, inputFileStd, outputFile)
        #ocmd='addcol -before 1 MATCHID $0' 
        #ocmd='addcol -before 1 MATCHID $0' fixcols=all 

            
        print cmd
        print os.system(cmd)

In [ ]:
%%time 

# For u-band, we will also match the previously GALEX- *and* GAIA EDR3-matched catalog with the FGCM standard stars...

if do_match_des_fgcm_aper8: 
    
    subdir = """DES_%s""" % (tag_des)

    tmpdir = os.environ['TMPDIR']
    band = 'u'

    inputFileObs = """../RawData/y6a1_rawdata.sorted.trimmed.y6_galex_gaiaedr3.%s.fits""" % (subdir, band)
    inputFileStd = '/data/des40.a/data/dtucker/Y6A1/StdStars/Y6A1_FGCM_V2_1_2_2_ALL_STARS.sorted.fits'
    outputFile = """../RawData/y6a1_rawdata.sorted.trimmed.y6_galex_gaiaedr3_fgcm_aper8.%s.fits""" % (subdir, band)
        
    cmd = """%s -Djava.io.tmpdir=%s -Xmx20000m tmatch2 matcher=sky find=best1 join=all1 params=1.0 in1=%s ifmt1=fits values1="RA_in DEC_in" in2=%s ifmt2=fits values2="RA DEC" out=%s ofmt=fits""" % (stilts_exe, tmpdir, inputFileObs, inputFileStd, outputFile)
    #ocmd='addcol -before 1 MATCHID $0' 
    #ocmd='addcol -before 1 MATCHID $0' fixcols=all 

            
    print cmd
    print os.system(cmd)

**DECADE:**

In [ ]:
%%time 

if do_match_decade_fgcm_aper8: 
    
    subdir = """%s""" % (tag_decade)

    for band in bandList:
        
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""

        tmpdir = os.environ['TMPDIR']

        inputFileObs = """../RawData/y6a1_rawdata.sorted.trimmed.y6_gaiaedr3.%s.fits""" % (subdir, band)
        inputFileStd = '/data/des40.a/data/dtucker/Y6A1/StdStars/Y6A1_FGCM_V2_1_2_2_ALL_STARS.sorted.fits'
        outputFile = """../RawData/y6a1_rawdata.sorted.trimmed.y6_gaiaedr3_fgcm_aper8.%s.fits""" % (subdir, band)
        
        cmd = """%s -Djava.io.tmpdir=%s -Xmx20000m tmatch2 matcher=sky find=best1 join=all1 params=1.0 in1=%s ifmt1=fits values1="RA_in DEC_in" in2=%s ifmt2=fits values2="RA DEC" out=%s ofmt=fits""" % (stilts_exe, tmpdir, inputFileObs, inputFileStd, outputFile)
        #ocmd='addcol -before 1 MATCHID $0' 
        #ocmd='addcol -before 1 MATCHID $0' fixcols=all 

            
        print cmd
        print os.system(cmd)

In [ ]:
%%time 

# For u-band, we will also match the previously GALEX- *and* GAIA EDR3-matched catalog with the FGCM standard stars...

if do_match_decade_fgcm_aper8: 
    
    subdir = """%s""" % (tag_decade)

    tmpdir = os.environ['TMPDIR']
    band = 'u'

    inputFileObs = """../RawData/y6a1_rawdata.sorted.trimmed.y6_galex_gaiaedr3.%s.fits""" % (subdir, band)
    inputFileStd = '/data/des40.a/data/dtucker/Y6A1/StdStars/Y6A1_FGCM_V2_1_2_2_ALL_STARS.sorted.fits'
    outputFile = """../RawData/y6a1_rawdata.sorted.trimmed.y6_galex_gaiaedr3_fgcm_aper8.%s.fits""" % (subdir, band)
        
    cmd = """%s -Djava.io.tmpdir=%s -Xmx20000m tmatch2 matcher=sky find=best1 join=all1 params=1.0 in1=%s ifmt1=fits values1="RA_in DEC_in" in2=%s ifmt2=fits values2="RA DEC" out=%s ofmt=fits""" % (stilts_exe, tmpdir, inputFileObs, inputFileStd, outputFile)
    #ocmd='addcol -before 1 MATCHID $0' 
    #ocmd='addcol -before 1 MATCHID $0' fixcols=all 

            
    print cmd
    print os.system(cmd)

In [ ]:
# Stop automated execution beyond this point...
raise StopExecution

## 10. Sandbox

In [ ]:
df_merge['u'].plot('RA', 'DEC', kind='scatter')

In [ ]:
df_merge['u'].RA.size

In [ ]:
#df_deszp

df_deszp_345_deep = pd.merge(df345_deep, df_deszp, \
                        on=['EXPNUM','CCDNUM'], \
                        how='left', \
                        suffixes=('','_y')).reset_index(drop=True)

df_deszp_345_deep[(df_deszp_345_deep['MAG_ZERO'].isnull() == False)]

In [ ]:
# DES footprint coordinates...
#  Read in round13-poly.csv file...
polygonDataDF = pd.read_csv('/usrdevel/dp0/dtucker/GitHub/SOAR_Goodman_ObsWebPage/data/round13-poly.csv')
ra_des = polygonDataDF['radeg']
dec_des = polygonDataDF['decdeg']

fig = plt.figure(figsize=(8,6))
#ax = fig.add_subplot(111, projection="mollweide")
ax = fig.add_subplot(111, projection="aitoff")

ra = df_deszp_345_deep.RA_CENT
dec = df_deszp_345_deep.DEC_CENT
ax.scatter(np.radians(ra), np.radians(dec),c='b')
           
# Plot sky distribution of each set of candidates...
# GW coordinates...
#for expnum in uniqExposureList:
#    df5_deep_expnum = df5_deep[(df5_deep.EXPNUM == expnum)]
#    ra5_deep = df5_deep_expnum.RA_CENT
#    dec5_deep = df5_deep_expnum.DEC_CENT
#    ax.scatter(np.radians(ra5_deep), np.radians(dec5_deep), c='b') # GW candidates...

plt.plot(np.radians(ra_des), np.radians(dec_des), 'r-')  # DES footprint outline...
plt.grid(True)
plt.title('DECADE u-band Data for DES Year 6 Deep Fields')

In [ ]:
inputFileName = '../for_calib/exposures_COSMOS_u_COADD_TRUTH_DESonly.npy'
data = np.load(inputFileName,allow_pickle=True)
print data
print len(data)

In [ ]:
inputFileName = '../for_calib/exposures_COSMOS_u_COADD_TRUTH_DESplsDECADE.npy'
data = np.load(inputFileName,allow_pickle=True)
print data
print len(data)

In [ ]:
%%time

query = \
        """
        SELECT * 
        FROM Y6A2_CATALOG
        WHERE ROWNUM < 10
        """

query = \
        """
        SELECT DISTINCT tag 
        FROM Y6A2_PROCTAG
        WHERE ROWNUM < 10
        """

#query = \
#        """
#        SELECT * 
#        FROM Y6A2_FILE_ARCHIVE_INFO
#        WHERE ROWNUM < 10
#        """

tag_eff = 'Y6A1_FINALCUT'
query = \
        """
        select c.expnum, c.ccdnum, 
               'https://desar2.cosmology.illinois.edu/DESFiles/desarchive//'||fai.path||'/'||c.filename||fai.compression as clink 
        from Y6A2_PROCTAG t, Y6A2_CATALOG c, Y6A2_FILE_ARCHIVE_INFO fai 
        where t.tag='%s' and 
              t.pfw_attempt_id=c.pfw_attempt_id and 
              c.filetype='cat_finalcut' and 
              c.filename=fai.filename and 
              rownum < 10
        order by expnum, ccdnum
        """ % (tag_eff)

    
    

# Query image filepaths from NCSA DESSCI db...

if True:

    print 'Query image filepaths from NCSA DESSCI db...'

    # The query...
    #  See:  https://sites.google.com/view/delve-survey/data?pli=1&authuser=1
    query = \
        """
        with 
        images as 
            (select i.expnum, i.ccdnum, i.band, 
                    racmin, racmax, deccmin, deccmax, 
                    'https://desar2.cosmology.illinois.edu/DESFiles/desarchive/'||fai.path||'/'||i.filename||fai.compression as ilink 
             from Y6A2_PROCTAG t, Y6A2_IMAGE i, Y6A2_FILE_ARCHIVE_INFO fai
             where t.tag='%s' and 
                   t.pfw_attempt_id=i.pfw_attempt_id and 
                   i.filetype='red_immask' and 
                   i.filename=fai.filename 
             order by expnum, ccdnum),  
        catalogs as 
            (select c.expnum, c.ccdnum, 
                    'https://desar2.cosmology.illinois.edu/DESFiles/desarchive/'||fai.path||'/'||c.filename||fai.compression as clink 
             from Y6A2_PROCTAG t, Y6A2_CATALOG c, Y6A2_FILE_ARCHIVE_INFO fai 
             where t.tag='%s' and 
                   t.pfw_attempt_id=c.pfw_attempt_id and 
                   c.filetype='cat_finalcut' and 
                   c.filename=fai.filename 
             order by expnum, ccdnum) 
        select images.expnum as expnum, images.ccdnum as ccdnum, 
               band, racmin, racmax, deccmin, deccmax, ilink, clink 
        from images, catalogs 
        where images.expnum = catalogs.expnum and 
              images.ccdnum = catalogs.ccdnum
        order by expnum, ccdnum
        """ % (tag_eff, tag_eff)
    
    print query

    # Query NCSA DESSCI db...
    connection=ea.connect('dessci')
    df0_dessci = connection.query_to_pandas(query)
    connection.close()

    # Output results to CSV file...
    output_file = os.path.join(output_dir, 'DECADE_FINALCUT.expimage_paths.csv')
    print output_file
    df0.to_csv(output_file, index=False)

In [ ]:
df0_dessci

In [ ]:
# DES footprint coordinates...
#  Read in round13-poly.csv file...
polygonDataDF = pd.read_csv('/usrdevel/dp0/dtucker/GitHub/SOAR_Goodman_ObsWebPage/data/round13-poly.csv')
ra_des = polygonDataDF['radeg']
dec_des = polygonDataDF['decdeg']

fig = plt.figure(figsize=(8,6))
#ax = fig.add_subplot(111, projection="mollweide")
ax = fig.add_subplot(111, projection="aitoff")

ra = df0_dessci[df0_dessci.BAND=='u'].RACMIN
dec = df0_dessci[df0_dessci.BAND=='u'].DECCMIN
ax.scatter(np.radians(ra), np.radians(dec),c='b')
           
# Plot sky distribution of each set of candidates...
# GW coordinates...
#for expnum in uniqExposureList:
#    df5_deep_expnum = df5_deep[(df5_deep.EXPNUM == expnum)]
#    ra5_deep = df5_deep_expnum.RA_CENT
#    dec5_deep = df5_deep_expnum.DEC_CENT
#    ax.scatter(np.radians(ra5_deep), np.radians(dec5_deep), c='b') # GW candidates...

plt.plot(np.radians(ra_des), np.radians(dec_des), 'r-')  # DES footprint outline...
plt.grid(True)
plt.title('DESSCI u-band Data for DES Year 6 Deep Fields')

In [ ]:
query = \
        """
        select t.pfw_attempt_id, qa.expnum 
                from Y6A2_PROCTAG t, Y6A2_QA_SUMMARY qa 
                where t.pfw_attempt_id=qa.pfw_attempt_id and tag='%s'
        """ % (tag_eff_des)
print query

# Query NCSA DESSCI db...
connection=ea.connect('dessci')
df_tmp = connection.query_to_pandas(query)
connection.close()

df_tmp

In [ ]:
query = \
        """
        select i.pfw_attempt_id, i.band, i.expnum, i.ccdnum, 
               i.rac1, i.decc1, i.rac2, i.decc2, i.rac3, i.decc3, i.rac4, i.decc4, 
               i.ra_cent, i.dec_cent, i.racmax, i.racmin, i.crossra0 
        from Y6A2_IMAGE i 
        where i.pfw_attempt_id=2085957 and 
              i.filetype='red_starflat' and 
              i.expnum=507126
        order by i.expnum, i.ccdnum
        """
query = \
        """
        select i.*
        from Y6A2_IMAGE i 
        where i.pfw_attempt_id=2085957 and 
              i.expnum=507126
        order by i.expnum, i.ccdnum
        """
print query

# Query NCSA DESSCI db...
connection=ea.connect('dessci')
df_tmp = connection.query_to_pandas(query)
connection.close()

df_tmp

In [ ]:
query = \
        """
        select distinct i.filetype 
        from Y6A2_IMAGE i 
        order by i.filetype
        """
print query

# Query NCSA DESSCI db...
connection=ea.connect('dessci')
df_tmp = connection.query_to_pandas(query)
connection.close()

df_tmp

In [ ]:
query = \
        """
        select distinct i.filetype 
        from PROD.IMAGE i 
        order by i.filetype
        """
print query

# Query NCSA DESSCI db...
connection=ea.connect('desoper')
df_tmp = connection.query_to_pandas(query)
connection.close()

df_tmp